In [ ]:
Objective: Create a cohort of PLWH replicating the work of the HIV subdomain team in N3C

# Queries

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by conditions
dataset_38981770_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
36687122,
4008081,
37017655,
37017261,
4347288,
36674252,
37017549,
37017071,
37017446,
37017279,
37017132,
42536591,
4320032,
37017318,
37017124,
37017209,
4340791,
37017244,
37017580,
37017278,
4314426,
37017456,
37017595,
37017284,
3654950,
4092686,
37017586,
36714339,
1340516,
37017093,
37017550,
4171124,
4225193,
4087604,
37017424,
37017094,
439727,
37019055,
4236860,
606040,
37018721,
36715476,
37017246,
37017106,
42536592,
37017108,
4087603,
37018711,
37018063,
37017425,
42536596,
37017092,
37017248,
37019034,
3654900,
37017243,
37017276,
37017294,
37019042,
4267414,
432554,
37017247,
606047,
45757132,
37116831,
4241530,
4253472,
37017319,
36674254,
42536590,
45769864,
37017210,
37017453,
37017262,
37017282,
4201627,
4171125,
3654682,
4128060,
37017126,
4239722,
37019058,
37019052,
42539031,
37017442,
36716524,
37017579,
4047624,
37017295,
4048033,
37017265,
4124361,
46284256,
37017259,
37018935,
37018755,
43531586,
4262297,
37017454,
37017266,
4161950,
37017254,
37017285,
42536594,
37017455,
37017457,
4180254,
37017652,
37017283,
42536595,
37017249,
37017125,
37017082,
37017112,
45757102,
42536593,
36674253,
37018714,
37116830,
37017260,
42538959,
37017296,
4172009,
37017320,
42538960,
37017263
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_38981770_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_38981770",
  "condition_38981770_*.csv")
message(str_glue('The data will be written to {condition_38981770_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_38981770_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_38981770_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_38981770_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), condition_type_concept_name = col_character(), stop_reason = col_character(), visit_occurrence_concept_name = col_character(), condition_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), condition_status_source_value = col_character(), condition_status_concept_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_38981770_condition_df <- read_bq_export_from_workspace_bucket(condition_38981770_path)

dim(dataset_38981770_condition_df)

head(dataset_38981770_condition_df, 5)

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by lab measurements
dataset_89725246_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
3026208,
3014884,
3017581,
4260330,
3025457,
3027554,
3026601,
4136494,
1175921,
3013180,
3031311,
40761081,
3034894,
3039421,
3032441,
3032957,
3017675,
3047359,
3014987,
3011044,
21491518,
3029235,
3048506,
3018408,
21492657,
4178288,
3034919,
43533928,
3024224,
3008301,
36304897,
3007417,
21494795,
46236978,
3015301,
21491520,
3036712,
3025164,
3013906,
3010021,
3030248,
45770655,
3042359,
4297026,
3034910,
3011325,
3015726,
36304535,
40490396,
3032045,
46236073,
3020647,
3022362,
3002868,
3042221,
3034881,
46235448,
3043306,
3032209,
4163881,
3030461,
3031839,
3004040,
3009341,
3015015,
3006608,
1175266,
3031110,
4326419,
4313584,
3005072,
3052999,
3013854,
40761080,
3042627,
44816748,
3011323,
21492993,
3014796,
3017286,
36203506,
3034979,
3006593,
44816747,
42529218,
4160613,
3039772,
40760301,
3016870,
40761084,
3032019,
3013123,
40762510,
3018426,
3003974,
36204239,
3043388,
3034301,
3012693,
46236074,
3030607,
3043283,
3038100,
3038207,
40765193,
46236976,
3037648,
3012160,
40489303,
3031607,
3034556,
3032527,
3010290,
3029093,
3013230,
36203852,
21491516,
3040587,
3046329,
1176285,
3039289,
4304803,
40488332,
3003852,
3046775,
40761083,
3032617,
3028845,
3019661,
3018154,
3031537,
43533927,
3042841,
1001749,
3032936,
1175878,
3010358,
3049147,
40761994,
3006826,
3003918,
21492985,
3044184,
3046636,
4176098,
3024449,
21492658,
36304503,
3000685,
3017385,
3028734,
4103459,
3027743,
3022061,
4036366,
4205734,
3048528,
3032684,
42527815,
3020089,
3011236,
3032427,
44790037,
3001000,
3027347,
3028148,
3021464,
4038853,
3010074,
3001843,
21491519,
3033479,
3036739,
3036139,
36203192,
3015190,
4036548,
3020778,
3032965,
3035962,
21493867,
3005467,
3031672,
3027287,
3027766,
4214450,
3039234,
3036399,
36306193,
3042661,
46236375,
3032786,
3020171,
36303628,
44816746,
3033964,
3034544,
3010747,
3043586,
3014347,
21493662,
40771875,
3043436,
42868715,
40762145,
3032606,
3031378,
40765206,
3052183,
3037274,
3040430,
3029801,
1175103,
40771495,
3031382,
3025705,
3052357,
3031319,
3032152,
36203199,
3032728,
3030101,
3031080,
1175398,
3031527,
4018596,
3051555,
44792202,
21493664,
44816745,
3040137,
44790057,
4137604,
3045126,
3032756,
3015682,
4038852,
3037935,
40760007,
3044493,
21493663,
4039702,
3035481,
40763960,
4201046,
36204237,
40761082,
36305228,
1175139,
4038851,
3036414,
3042525,
43055394,
3020767,
3043837,
3042577,
42529489,
3022655,
3026532,
42529488,
3048535,
3037337,
3050934,
46235355,
3006323,
3012733,
36204238,
3004365,
43055393,
3031067,
3032287,
42870583,
46236977,
4035171,
3001240,
3036417,
46236979,
3035725,
3048236,
3025907,
21494684,
3021850,
36304555,
4040194,
3009280,
3035836,
3053246,
3033937,
3008121,
42529487,
3046956,
3000086,
3040890,
1176439,
36304523,
3007921,
40771874,
3000170,
3007136,
3008039,
3047064,
43533684,
3050623,
21491517,
3007679,
3045827,
3007624,
3045160,
3051000,
1175561,
3033106,
46272018,
3035629,
36305086,
3039370,
36203237,
3044830,
3008716,
43533683,
21493878
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_89725246_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_89725246",
  "measurement_89725246_*.csv")
message(str_glue('The data will be written to {measurement_89725246_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89725246_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_89725246_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_89725246_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89725246_measurement_df <- read_bq_export_from_workspace_bucket(measurement_89725246_path)

dim(dataset_89725246_measurement_df)

head(dataset_89725246_measurement_df, 5)

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by drugs_query 1
dataset_74970258_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (
41208473,
44045833,
45892739,
41201623,
43026166,
19081175,
36061908,
21033712,
41260971,
36249788,
21110708,
36061971,
21023723,
43169082,
45892214,
21026616,
44062950,
41257682,
21121799,
44128617,
43177133,
40730818,
36261256,
43661062,
41209226,
40919110,
36257452,
41228124,
43146908,
43022435,
44159913,
44058013,
40724596,
36227427,
40753349,
2052801,
21121991,
36062188,
36061611,
35858572,
41252913,
36248761,
41259821,
40709680,
21131575,
42706894,
41011265,
41199396,
19006257,
40961370,
42918908,
41288583,
35862031,
43204732,
44042631,
44114706,
36894603,
43177134,
35604225,
43022451,
42683461,
40943476,
21063033,
40882846,
41103894,
1781410,
44089547,
2052767,
43160868,
42708096,
36226561,
43268522,
40875590,
43187466,
43023778,
21121987,
1511082,
21043415,
19102831,
35412092,
40720869,
41381586,
43202075,
36785881,
36890547,
43158209,
40097200,
41273222,
36419264,
43144917,
36225762,
21075763,
41046666,
36276926,
36891879,
40847307,
45775747,
43031551,
44171834,
41071598,
21053215,
40741180,
19127304,
1781407,
21173974,
41226698,
43145150,
41245640,
44205180,
19079870,
36277507,
21033496,
44167743,
43786180,
36242594,
41133284,
36225757,
43032210,
21024319,
35768846,
782823,
43134209,
41290902,
41072413,
35862178,
1738171,
40710120,
41264435,
21147823,
40746925,
1748955,
35860837,
36248986,
44061790,
2031819,
41412005,
40241982,
36505730,
42707688,
21141532,
42655678,
40162285,
44175087,
21101476,
40746912,
36882760,
21131569,
44108421,
21072949,
43174017,
995225,
43294619,
43026171,
1736996,
19125940,
35862180,
21055213,
42902283,
44161262,
44106041,
41190787,
36249786,
36239227,
43143367,
36062382,
40990913,
35606589,
43268385,
35788645,
2053067,
43158158,
43212907,
1748987,
35752148,
40874083,
35603888,
36784667,
40049811,
35606581,
36782726,
43138382,
40885494,
36785878,
43713465,
1592273,
41304479,
40930058,
43291040,
37592144,
19098330,
41150671,
1718489,
43039286,
40996625,
40714440,
19038141,
19091081,
36238740,
19122306,
43293565,
21063039,
40821843,
43526311,
40736230,
43149294,
36788885,
40978758,
43138732,
36062179,
44076530,
36242591,
43026156,
21171507,
41169651,
36061608,
43256643,
43180130,
43155130,
35862020,
36779743,
21112306,
40950313,
43133976,
21120401,
35200460,
36889217,
782826,
36245658,
40730827,
41136273,
41435736,
41294029,
21100250,
41101949,
21171050,
36249080,
43187468,
44178254,
41404350,
40746917,
43522868,
44205225,
19124377,
35859795,
35411785,
41135304,
41187846,
36248699,
40971747,
43151360,
19123440,
19022896,
19045249,
40918006,
43180074,
41197044,
42656090,
36224276,
21157929,
42543870,
43163199,
43200283,
43180173,
43189263,
41502054,
21118438,
43185065,
21131570,
40165045,
35773215,
36271282,
43026751,
36784670,
19125469,
2052591,
44196810,
21149333,
36238998,
782836,
41270318,
40733519,
21112608,
41221906,
21072940,
36784660,
19102250,
35415360,
21147041,
40851568,
36404012,
21069538,
21144478,
21072947,
43189254,
36062303,
37592830,
44186404,
43026143,
19113114,
21033498,
40930180,
40879056,
41430784,
35860787,
21122327,
19027669,
35860452,
44045755,
1511230,
40855382,
43219711,
21131574,
1738162,
41168002,
44044297,
43144916,
40234303,
41103003,
41435748,
44129866,
42918909,
19125938,
37592828,
2052718,
41089766,
43134211,
42874705,
43195889,
44111050,
44121653,
41003135,
43133085,
41239851,
43270867,
40947772,
21023722,
21151341,
43261494,
2053047,
41306667,
35200465,
21069464,
36248702,
36783451,
42927739,
42927758,
36061604,
40010553,
19121070,
40730819,
42708114,
21092413,
45892974,
41037971,
2011837,
43156112,
782438,
42874953,
40928353,
41234056,
36074451,
40917580,
21145007,
43860998,
41280887,
41083974,
41132956,
43211097,
41435058,
1710612,
19129965,
1758537,
44067769,
43146977,
44119366,
40825250,
36249552,
36219237,
35858570,
35415358,
1724993,
44065245,
21154304,
21134119,
21169052,
36407004,
2052770,
41179980,
964218,
35862764,
40238459,
41206774,
43212951,
43149972,
41135083,
21026702,
21164103,
19122565,
41013502,
41432040,
42927714,
41165668,
36062174,
35750794,
36420271,
36783449,
40127987,
21151898,
35760526,
41054831,
36788886,
40720638,
43522795,
43258430,
782839,
40955101,
2052765,
41280885,
35743671,
43589260,
36261384,
35858569,
35411994,
40945509,
42967484,
19065536,
1725064,
36267870,
35410895,
19041909,
36249079,
41315122,
40948924,
44048810,
40730838,
45892117,
41464361,
21082608,
40909781,
1725063,
43515541,
21082831,
35858553,
21102497,
43840728,
35862767,
43290082,
44106040,
43291041,
40977858,
19123901,
40753344,
1592433,
19124374,
2052776,
35409733,
43262883,
40918004,
43211100,
41435685,
40945667,
44057005,
44086573,
41290665,
44070910,
40708526,
42543878,
40886688,
41199572,
36407765,
44071629,
1738202,
35859812,
21033960,
40976777,
40720643,
35408211,
36785885,
41164638,
1727223,
36259364,
43143990,
587054,
43193750,
36219270,
40032839,
40874082,
19047928,
21121234,
21085486,
36223704,
21026709,
36061898,
43298482,
41123761,
19102469,
35411824,
21063034,
41041200,
36257626,
782827,
40746926,
42927761,
964018,
41365231,
40080338,
41292362,
44127513,
43169046,
43026758,
36062290,
35757504,
40095221,
21023721,
21167670,
36249782,
36242993,
41321319,
41148899,
44086143,
44178717,
19102470,
21032701,
41159231,
19125943,
43037164,
41011683,
43189252,
21069539,
36503871,
21072950,
19120186,
36880598,
43162536,
40898935,
41312203,
40978751,
40166596,
43146909,
43140379,
41211615,
40950315,
21053213,
41411931,
36074449,
41117641,
1704183,
1704217,
41250074,
19015459,
43191063,
41469748,
41197042,
40864786,
1756831,
44096044,
44120244,
40844389,
42874225,
42967486,
36404666,
19078497,
44188518,
21102496,
42705416,
21157930,
43032456,
1738165,
40708525,
43296720,
41435605,
1710659,
44205179,
43285658,
40162290,
41107092,
40720649,
43037165,
45892744,
21092417,
41106649,
995224,
41412003,
994687,
43294113,
43026152,
1510226,
42927736,
40947776,
36272454,
41261212,
41434399,
43678859,
41280882,
19113609,
36275044,
36890656,
36219268,
37592142,
42543875,
36784672,
21075859,
43178347,
41299919,
41041201,
44040584,
21030087,
40712269,
19048073,
43283254,
21082603,
41195630,
40927355,
40241981,
35861975,
41232113,
21154397,
36257199,
41323443,
42705554,
21159260,
41009206,
40977854,
36261828,
21065836,
2052738,
44028387,
43026168,
40051188,
43696024,
40160002,
41088350,
36062293,
40932068,
41256602,
2052763,
35412534,
43169042,
43167287,
1748989,
40956688,
21164210,
42941332,
43178343,
35774329,
36062284,
41190788,
41235209,
42927724,
2052737,
43804510,
21023938,
43202117,
41026831,
21134575,
35861966,
21051386,
43166158,
1736999,
1718486,
21053214,
41235195,
21164209,
19102211,
36061613,
36419263,
35743804,
35770801,
43178349,
44167259,
36062299,
41040936,
36062297,
35200457,
35415364,
41136963,
35414451,
40706773,
41302143,
1711524,
35756375,
43272621,
40976102,
40979945,
40142130,
43588591,
1736923,
41228645,
43189261,
40749146,
1756840,
41430636,
41221905,
43022445,
44107532,
2011766,
36272121,
35858556,
41435606,
41117647,
44127860,
43288724,
40909013,
43186772,
35750352,
35860458,
37594090,
40730843,
43659804,
40916369,
2053071,
40835037,
21034051,
40979421,
21161276,
43195890,
41222274,
36062291,
42705918,
41324936,
44185709,
41101257,
35771278,
35414412,
42927749,
41404323,
793027,
36062298,
41006884,
43858805,
43587495,
21033495,
36785883,
43026157,
21108714,
19088562,
40733278,
35773217,
40720766,
43155129,
43206300,
1758539,
36226566,
36214321,
41228646,
43821524,
1746244,
43277959,
44087733,
43178344,
41288585,
41148915,
41201042,
36220596,
43200288,
44114794,
41250072,
44168094,
36236511,
2053060,
44127082,
36242596,
35858575,
35861965,
36220376,
41117644,
36277450,
42874220,
40844391,
41433119,
35862029,
41195247,
43022448,
43160866,
40897794,
41312204,
40733289,
40885492,
43605578,
43133974,
44037965,
41430634,
793028,
35858561,
40008594,
40898933,
43158447,
1718462,
35767613,
44183046,
1748985,
42927716,
36235768,
43182790,
40992055,
43022460,
41207012,
41321875,
35860603,
1511233,
21134572,
43180400,
44061791,
21033493,
21102875,
35752147,
43732231,
44121980,
42963129,
2052736,
44204772,
43028257,
44164477,
36248168,
1736971,
43136093,
2052748,
19078496,
41435735,
36897682,
40855385,
44091881,
1715476,
41272807,
21169144,
36889986,
43039287,
41211441,
43202118,
994686,
40874080,
41471034,
35743803,
43028263,
21072948,
2047892,
35862023,
35200454,
2050980,
35860840,
40975405,
21060948,
44166954,
42875456,
41226850,
782648,
40162291,
2053040,
40925729,
43034923,
35768845,
45892978,
44167258,
41170647,
21082604,
35200466,
43156338,
36895800,
2011839,
35410515,
35200450,
21132158,
40855383,
40945664,
41211425,
19129167,
1725065,
21121798,
41042360,
43204906,
19010081,
36222515,
35200464,
35858565,
21104934,
41430811,
41111364,
43189265,
41277370,
43263864,
40825359,
41197948,
40916125,
21100338,
44095715,
21080573,
40996626,
1715473,
21161282,
35862593,
40842858,
42965765,
1560081,
40937841,
19102236,
41288584,
21124650,
35771931,
40730836,
21033503,
44785499,
43180399,
40733288,
43178348,
41038514,
35862011,
43146976,
36061970,
44182584,
35146811,
42941336,
41435384,
40733285,
40855106,
42705418,
41156729,
43032450,
41280884,
41435662,
21041342,
41136534,
21140070,
40730828,
40875589,
41239951,
44171218,
43146978,
42927742,
40864787,
43180174,
43023780,
36784669,
793026,
21131779,
40706774,
41165671,
21169055,
36404479,
42705925,
40171778,
43167286,
21039999,
35861030,
36217381,
19116946,
43696022,
35771967,
41088351,
43171965,
44171220,
44082743,
1592435,
36257399,
41435638,
41096914,
43200295,
44101047,
43156337,
21100884,
21149923,
36062294,
41006780,
43026161,
35136179,
41411994,
40972100,
40720634,
21154084,
36062159,
40730815,
43261490,
43026752,
35860796,
36074476,
40221116,
41042359,
2052749,
21139717,
19102201,
42705927,
2031631,
2052731,
21161289,
36243648,
35605554,
35606580,
41290901,
40051184,
43178107,
19042159,
40051187,
43211099,
21043418,
44216219,
44171511,
42705414,
1718279,
44205300,
41010082
)            
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_74970258_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_74970258",
  "drug_74970258_*.csv")
message(str_glue('The data will be written to {drug_74970258_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_74970258_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_74970258_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_74970258_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_74970258_drug_df <- read_bq_export_from_workspace_bucket(drug_74970258_path)

dim(dataset_74970258_drug_df)

head(dataset_74970258_drug_df, 5)

In [ ]:
# Rename df
df_drug1 <- dataset_74970258_drug_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by drugs_query 2
dataset_74970258_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (
1511090,
40842857,
36224278,
36272195,
35746160,
43191283,
44096910,
21033494,
21171239,
41178959,
1769415,
36230093,
40975511,
43642065,
44175550,
36263870,
21171053,
41166551,
45892973,
36237503,
44178253,
35764659,
40835547,
35860789,
41003134,
2052766,
35861989,
35861968,
41205554,
44177533,
36061952,
42927715,
21169667,
43180131,
21140072,
40746915,
35861972,
42941334,
41117646,
36409071,
41294032,
36411418,
21104772,
41435786,
36061599,
36274244,
43269333,
43147024,
37592829,
2052750,
2053075,
35860790,
42927721,
43151359,
40130705,
35860780,
35860839,
45774772,
40720283,
43187422,
43145152,
43623448,
35862766,
21051767,
44178718,
21073134,
35859811,
40720280,
40917581,
43560387,
43785857,
44182209,
36507139,
35861964,
42874956,
44161263,
43140377,
44174952,
43297764,
21060951,
35751888,
43156335,
1710616,
36217482,
40753356,
43277958,
43025640,
41435809,
36248535,
44163577,
44113270,
41367239,
40965072,
21134576,
43749411,
35411828,
40724591,
36061596,
41382002,
21173675,
21112314,
44170968,
35748011,
43175843,
43202122,
41041202,
2052721,
41435790,
41176026,
44101098,
1715472,
44170585,
21147949,
41086183,
41404313,
40051182,
40097505,
2052768,
41288218,
21075762,
35606585,
21065614,
41252912,
41190329,
43206860,
41197949,
43209113,
40724604,
41069115,
36788487,
21041343,
43189028,
782846,
42709025,
40937840,
19112067,
42875915,
43174803,
964008,
19131468,
44185984,
41290564,
40921138,
21082599,
2052740,
43153842,
19047768,
21085166,
35772093,
21080574,
44186353,
43026163,
1710654,
19079684,
41101256,
43191020,
43028258,
36061974,
35859799,
43660865,
40720770,
36219234,
41282493,
40742423,
36261430,
43200291,
41136962,
40978759,
36788485,
21142103,
41191169,
43218581,
40071866,
41290898,
35862015,
44180760,
44036307,
35606582,
40720282,
41237236,
43022432,
43022444,
41177701,
21131578,
41136544,
41463278,
21112109,
36788473,
1718478,
41100704,
41139245,
21161280,
43642243,
41325360,
35862763,
21051155,
36236162,
43839615,
43154372,
35411296,
41168140,
1738167,
43168991,
43140836,
35860795,
21032700,
41072090,
43202073,
35752146,
43167284,
36062292,
21138091,
42927717,
41258928,
1769413,
41435782,
1781438,
35861997,
35862598,
45892742,
21061185,
41071596,
35860788,
41293605,
2052719,
43189029,
19124992,
1711526,
40730826,
37593310,
40825633,
41040308,
36061594,
21053218,
21056141,
40945268,
41303337,
43146979,
1510224,
41135084,
41186293,
40978755,
41062397,
40959675,
35200468,
42874783,
43200286,
43143988,
2052786,
43265387,
2011843,
21026526,
43022221,
19082373,
43519084,
40220794,
43515342,
43031552,
1769414,
43031557,
35860785,
43200289,
36219267,
19084909,
21121986,
45892741,
41320002,
2050590,
36061904,
2052742,
40869878,
21171052,
36404313,
21030149,
36506462,
43840211,
40161653,
21092967,
41135306,
40885498,
40945117,
21149924,
40887121,
40975404,
40072742,
21112111,
42927704,
35771968,
41275457,
35604228,
44159282,
21063036,
44082744,
43180132,
43144915,
44050940,
44161036,
43288720,
21102307,
40921943,
19058557,
40241980,
36061901,
40724595,
37593313,
36222272,
40945669,
21055923,
42901583,
43169085,
41134412,
41008133,
35862602,
40924860,
42927735,
1511088,
36273917,
40140031,
36240291,
43261491,
43202367,
35764671,
45892977,
36259586,
21151824,
19125236,
19107920,
43803833,
41239853,
43146975,
36061953,
19131908,
42708121,
43209325,
35746161,
43298357,
35764670,
1718550,
41323598,
36061612,
41007597,
21092416,
42941327,
40971274,
36273748,
36239209,
35862012,
43609440,
41086366,
21082833,
21114740,
43184389,
41302840,
41236246,
40882156,
40838797,
41151164,
36412246,
40154238,
36783447,
36218528,
36061903,
40906480,
36778127,
41260983,
36269549,
43030473,
44041760,
1789455,
42927755,
43169047,
36784665,
44121505,
1724912,
43750550,
35605921,
36232127,
21032078,
41069222,
21121802,
21151560,
2011838,
36213369,
36236164,
43212957,
43180134,
40987956,
37592138,
21108650,
44115232,
46275632,
35858559,
43280466,
43822634,
41062398,
41149219,
36504283,
36243992,
43149975,
35768603,
44168401,
43189256,
35860786,
43022458,
43190967,
35860836,
43840730,
21023725,
36504352,
43184867,
42927722,
43588573,
36214475,
35770020,
44204751,
43210854,
21142013,
43177135,
40051162,
36065920,
40143572,
21053409,
21171237,
41312206,
40741181,
41042366,
40720871,
19079686,
41430792,
43200065,
36781988,
35862028,
21151343,
782652,
36062288,
35860779,
36785879,
21063041,
40730830,
40978424,
41378269,
41112903,
41290031,
36062161,
2052751,
40854226,
35605556,
21071497,
35156246,
41096442,
40947773,
40008593,
43172104,
42927718,
587311,
2053051,
45774770,
41169220,
43158206,
36268199,
43138383,
36891509,
1736925,
41103893,
40883376,
35860455,
41148917,
41105553,
40142304,
41064865,
40882848,
21171248,
1592436,
36788477,
41229015,
43037156,
1710656,
42874784,
21043413,
41260968,
40861829,
40896365,
36218352,
43733306,
37594088,
41075644,
2052788,
782832,
43714567,
36779000,
21056113,
44030939,
36230119,
21053663,
19124342,
44112432,
41260970,
40982471,
36061959,
40981883,
21141530,
40730840,
21083114,
41250075,
41435683,
41060965,
1704191,
36241911,
40914147,
1746248,
41304478,
41021878,
21134573,
964012,
19123900,
21151342,
41435283,
36403691,
35858562,
19103096,
1724918,
41312205,
1748960,
43840729,
36223705,
1738204,
21073128,
21023727,
36778174,
21149422,
35762188,
44067260,
43028260,
42705551,
21092420,
40979666,
41221488,
40720772,
1736997,
35859814,
40032845,
41135305,
36061906,
21124559,
21033961,
35862596,
44111895,
1703069,
40746918,
40945267,
40971261,
40724600,
21141531,
43145155,
782841,
21033713,
21141535,
21161281,
40720874,
43822108,
35860797,
2052779,
44182583,
41072412,
36061610,
42481363,
2053053,
44171512,
43156110,
21139474,
40720632,
44093004,
21043416,
41011687,
41435672,
40852101,
21085487,
43752639,
2053068,
43032546,
43176513,
36241914,
35751886,
37593206,
40080336,
43257628,
44174956,
41135085,
21102495,
43167285,
41294031,
35604229,
41462731,
35862466,
43166154,
41435822,
43178108,
21124558,
40058823,
43624656,
41435313,
44043480,
43785855,
41242524,
36061607,
41280883,
45892556,
19103099,
36061606,
21161291,
43026164,
41015711,
36893408,
21114741,
41101768,
1738135,
43138927,
35862770,
41072414,
41258929,
43193907,
40736229,
19081174,
43023782,
40753352,
41315123,
43134208,
44216221,
36238741,
21051153,
44174954,
35859824,
36275802,
44163693,
44090903,
44176187,
44041366,
44170738,
40058382,
41288216,
43143989,
35773212,
43166157,
43155132,
35862008,
43026145,
36884120,
41074119,
42874788,
19131469,
41123757,
41435652,
36412245,
41134413,
41200616,
36062165,
36062170,
41117412,
41027628,
41011257,
43155133,
43149848,
21072945,
44036777,
43199153,
41152923,
43136094,
36783255,
35764664,
19129166,
36897543,
40730823,
21159878,
44095150,
44093106,
36233245,
44163224,
41217621,
44075984,
43023784,
41029809,
1704244,
43022329,
35745445,
41096917,
35788655,
36229805,
35862184,
36058615,
2052774,
43039284,
41166553,
37593309,
41011686,
40746935,
36811149,
43298296,
1560084,
40733284,
40720631,
44174953,
41435634,
43022455,
21112113,
2053052,
43804528,
40853348,
41435789,
36406645,
44101877,
43171802,
1724862,
40720768,
40825040,
35756373,
36778172,
43206760,
40917579,
35862022,
42918911,
35768849,
782441,
40733282,
45775750,
41435756,
21039123,
21065947,
35862751,
43767981,
35860783,
44185539,
36896588,
40733287,
41166314,
42875780,
43022447,
35862182,
40720876,
43174223,
2052754,
40128015,
1789454,
40712271,
19086220,
36882438,
40916124,
43167283,
43133975,
45893034,
19047527,
40072743,
40882847,
19129163,
41163925,
43197769,
40919587,
2052720,
43858287,
1718488,
36249784,
21095931,
19103687,
2052798,
40730824,
43156113,
43204708,
36062169,
41065345,
35408398,
43804509,
45892740,
19127303,
1756841,
40854227,
40095224,
41292116,
41435760,
43154311,
43166153,
36783448,
43140378,
40912393,
36239640,
35415371,
41463975,
43804508,
41115960,
43149974,
1738164,
19047803,
782824,
40730825,
40875588,
43623814,
41039028,
43032449,
44178716,
41187845,
36788486,
35410525,
43166159,
19006864,
35411176,
41242323,
19098116,
19102557,
40906476,
44175551,
36074452,
35606590,
19038785,
40947774,
793031,
40945666,
43032451,
36880751,
19102768,
1717002,
40173165,
40885161,
36788481,
42708116,
2052775,
40742424,
21124682,
41235210,
21023930,
45892118,
1724863,
19048077,
41452312,
35859822,
36265550,
1718479,
42874785,
44159914,
43032443,
43151358,
35749855,
42706893,
21173887,
43696194,
19076448,
19100779,
41435747,
44032033,
21173980,
43293564,
41015448,
41322222,
19047924,
41165669,
43043804,
35860605,
43158210,
21056114,
40855384,
19131907,
43660369,
36226564,
40884570,
35760392,
589144,
36788492,
41072409,
36222518,
40915514,
40940281,
1738203,
40867643,
36273925,
41291858,
43189262,
41002684,
44178615,
43284816,
40712270,
40906484,
41007820,
41133092,
42480754,
1748957,
35605925,
40095137,
43022442,
1592434,
19040427,
40709688,
35751887,
35407590,
43642725,
41323446,
19038142,
35761434,
35862595,
36074483,
40724598,
36217379,
21053219,
35764666,
43165475,
40166032,
41096443,
41152356,
40720641,
36248987,
19025941,
782833,
44070129,
35756846,
41435795,
43146972,
42707832,
36243626,
40855790,
40992630,
35760285,
42683344,
41146009,
41101950,
21070915,
41013152,
35415320,
21161277,
21065488,
40720629,
35743669,
41010263,
43266354,
21131769,
35743809,
21119805,
36248762,
36218355,
21124658,
35860454,
36812938,
36062285,
40724594,
42967485,
41284219,
43209310,
19127305,
43030470,
35756374,
21079307,
43260893,
36249077,
36897515,
35862030,
36242880,
41292507,
41201053,
40049809,
36218348,
1748959,
43188104,
44205000,
21033502,
35862017,
36062183,
19122305,
40133801,
41291602,
36407764,
43255571,
21149331,
44216220,
41163253,
21036159,
21110079,
41262398,
41186294,
21132080,
35862597,
45892116,
43212906,
36215507,
36893697,
43212952,
19124373,
41235204,
43182762,
36062286,
1718276,
793025,
41010080,
1560083,
19097793,
21094945,
41402640,
21033505,
40855788,
43215815,
35760527,
35862170,
40916367,
43145151,
41435696,
43858285
)            
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_74970258_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_74970258",
  "drug_74970258_*.csv")
message(str_glue('The data will be written to {drug_74970258_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_74970258_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_74970258_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_74970258_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_74970258_drug_df <- read_bq_export_from_workspace_bucket(drug_74970258_path)

dim(dataset_74970258_drug_df)

head(dataset_74970258_drug_df, 5)

In [ ]:
# Rename df
df_drug2 <- dataset_74970258_drug_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by drugs_query 3
dataset_74970258_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (
21141526,
40147739,
44102552,
36788888,
40853343,
41246494,
41435655,
36062175,
44186403,
40746916,
36782730,
40851567,
21046356,
19006255,
43171801,
44120603,
35862027,
36227428,
40084800,
40709689,
36782729,
41470852,
43167290,
41258931,
43200294,
19081173,
36274113,
43210851,
43261495,
36243647,
35859821,
41290030,
2047893,
36270662,
1756872,
36238052,
36895786,
42874787,
43786025,
19047372,
43188105,
35764668,
19117688,
41230229,
43191284,
19125939,
36061969,
43272949,
35407037,
42927729,
43272464,
1787124,
36061902,
43295646,
35860604,
40058746,
44181824,
43152715,
40720278,
36508891,
43288112,
964020,
36218353,
43822110,
36062167,
42941326,
43840731,
40134707,
1718482,
42927756,
35858555,
41197045,
40831775,
44060472,
41065347,
43174018,
41041856,
43588572,
40936271,
36074479,
43272465,
40882849,
21143972,
42657242,
21171506,
40736231,
36782728,
1704214,
1560082,
1710281,
40963441,
43152088,
2053066,
41159218,
19123439,
19010237,
40978754,
44204787,
1724920,
40825985,
42874957,
41011256,
35862468,
40923765,
40909030,
36061958,
40975509,
43209921,
37593513,
44031003,
1738141,
36219736,
35754729,
42963128,
40854225,
21069606,
43026165,
44093107,
21072939,
37592141,
35861988,
43166156,
36062295,
1704190,
37593516,
36242994,
36270751,
42927701,
43169044,
43026142,
35859820,
1711523,
21161278,
1718485,
21063037,
36226581,
42708097,
41040935,
19042160,
41031267,
40165043,
41323436,
43283253,
36282088,
41034576,
36062302,
43023777,
40137867,
1718484,
43215247,
41000185,
43176520,
43767979,
21171593,
2052735,
40166595,
41011684,
21131577,
35861996,
36788484,
41213681,
43732778,
2052803,
44070911,
43031548,
36235769,
36242590,
35772958,
2053070,
36219266,
43039285,
43678367,
36219737,
40746922,
41103646,
2052796,
782843,
782649,
36271562,
19076443,
43750705,
40220797,
41221904,
43261445,
36879888,
36074478,
43714569,
1718460,
36061600,
44029600,
42874223,
21144298,
41517303,
41062394,
40706775,
43293563,
42927711,
19120798,
43156111,
40940776,
43180133,
1710316,
994580,
40838798,
43169084,
44082745,
43298529,
2052755,
41435780,
42875455,
21026209,
42656092,
43022434,
35861961,
40838933,
40977857,
41094046,
41117643,
41434479,
42927726,
19102177,
35415372,
19123898,
36247224,
44082598,
41105554,
40079020,
41040309,
19022920,
36219437,
43039288,
41435787,
21082605,
35861970,
40916363,
41197950,
35862181,
40720870,
41160014,
44086572,
43143987,
19020192,
45892557,
41259823,
41273279,
43031547,
43026169,
40905017,
19047371,
21094460,
36784661,
40916368,
41244664,
35859825,
41243902,
44062948,
36887027,
35415375,
43145154,
40947545,
43200293,
37593312,
43153881,
43037158,
21124656,
43190965,
43209920,
43288721,
41290906,
35412638,
41280886,
40749145,
36880255,
36420272,
21149567,
35858560,
21063737,
587802,
43142737,
40894207,
21144039,
36233244,
41034213,
44088533,
1711552,
21157922,
41242332,
43768533,
40948679,
35410146,
35742684,
40981428,
41404433,
40720633,
36248760,
43696348,
36886298,
35604224,
41096915,
44128543,
43205025,
40846862,
41096916,
35743811,
35410896,
21033962,
1710657,
40906481,
43190963,
41273206,
19015676,
43037323,
43255499,
40905019,
43588073,
36226580,
21051154,
41044255,
43135984,
2052456,
40171781,
782834,
43217790,
41470436,
36405247,
43174222,
35862024,
43022436,
40733279,
40720774,
1511232,
36778173,
41075645,
42481362,
41010081,
36886825,
43032448,
36061975,
21080806,
43768138,
41502682,
21081834,
21161290,
41021228,
43714019,
36506718,
35752143,
36074481,
41136532,
35410716,
40824780,
40720281,
36788489,
41221487,
35861028,
36061605,
42902282,
21124179,
19086161,
43158208,
40166033,
43258429,
36062287,
43031558,
43522495,
41029810,
41411960,
40147469,
40856852,
43197819,
40875585,
41043834,
2053059,
21161843,
782842,
43026147,
36061601,
44160177,
36894870,
44025922,
43840467,
40887119,
19022953,
36248169,
21092415,
964019,
35773213,
36783446,
42927754,
41197611,
35861967,
35767726,
41517299,
2053073,
19091082,
41252485,
36788482,
43193382,
40857297,
588291,
36788883,
42657241,
41242523,
43713866,
41135086,
43167282,
41119147,
19120644,
41292122,
21134474,
43022332,
44076524,
21105228,
40714441,
782835,
35603887,
41323699,
21140071,
36237141,
36244925,
44036361,
36509777,
36407409,
41144269,
36062164,
36784671,
43215246,
19133699,
41435615,
43208303,
19129077,
36788488,
35411297,
43043803,
19047926,
43206304,
41092533,
1715475,
40237542,
36781986,
41430646,
21102786,
44102439,
40746936,
40008592,
1718491,
21134574,
2053072,
40032844,
44186507,
43169081,
43212910,
41277153,
44171219,
40753351,
36262103,
41136035,
36237802,
40059335,
41259822,
36061907,
41260969,
43193749,
46275633,
41042791,
36273267,
36406644,
21043420,
19022954,
36217382,
19089436,
36219269,
44165999,
40847306,
43184873,
43659803,
41057982,
35760528,
42901582,
43858117,
43749412,
35764672,
41435620,
40139242,
43750000,
41078330,
40888608,
42927730,
40171822,
1704212,
40895018,
41298103,
44167686,
41038516,
43175881,
1718487,
43212950,
44037677,
42927752,
2050960,
35775061,
1710621,
35862175,
40853346,
43156339,
36061960,
43184387,
41199149,
42705926,
42927732,
41051695,
21151567,
21073417,
43031550,
19121244,
43167288,
41433224,
41229017,
42927744,
21171054,
43158205,
40008595,
41101948,
43678527,
40720767,
43169083,
42543877,
43749413,
2047894,
21043613,
36214927,
35409675,
40838799,
35862756,
21139473,
43714020,
36239208,
43151982,
40961372,
35862173,
42874221,
21161279,
42927734,
43141013,
41470980,
36218350,
41167762,
19103468,
41470790,
41430642,
41435616,
44181825,
21092609,
41230641,
40821764,
1511085,
44174336,
36222514,
21141734,
1718483,
21152066,
43285659,
41034212,
43035797,
19043586,
36062158,
35411467,
41432537,
43175926,
36812153,
21132159,
43732073,
40049810,
35859818,
36812767,
41200615,
19103085,
43030471,
41471173,
35764667,
19122308,
41168004,
21104802,
21134088,
40051161,
36262714,
43642247,
36788884,
21141533,
43283251,
40730814,
41094047,
2051327,
40730839,
41159230,
21059578,
43200284,
45892114,
2052795,
21034049,
37593514,
43261493,
40853351,
36061899,
36236271,
40913804,
36061897,
43184871,
43028261,
36062283,
41163252,
43840703,
41435804,
41412009,
36409073,
21134250,
21110299,
41165670,
41165666,
36258300,
41125213,
21134578,
36218349,
2053046,
35760283,
1711554,
21065838,
41148916,
43217925,
793032,
44125083,
43258428,
40730832,
36062176,
35862007,
1718480,
36257912,
40916365,
41038518,
41069783,
41096444,
35200462,
41470498,
1789431,
43160894,
36890046,
43166152,
40961373,
43133086,
2052783,
21105051,
44158931,
42657627,
41292123,
41452997,
19102500,
43212953,
41134972,
41011689,
44205445,
41287653,
21030129,
36219232,
41195801,
40916370,
43212954,
44127859,
40870013,
41088352,
1724830,
41229013,
36233143,
40878425,
35412057,
43025641,
41256603,
36263434,
40921162,
41301770,
35773216,
43255572,
43028262,
40720647,
40058822,
21154418,
36277225,
43660518,
35862025,
21112305,
35862188,
41045026,
21041344,
2053048,
43516046,
43786494,
35603890,
43174016,
40853347,
41128202,
21053220,
2053055,
40960187,
41262397,
964014,
44067448,
41464229,
40143571,
41323435,
36241910,
36226563,
35862186,
44159818,
35862018,
43202119,
21073135,
19047350,
43204733,
41469871,
41072178,
41010078,
35862183,
994581,
1781436,
44101875,
36241913,
40746930,
41065346,
40120325,
587328,
43641655,
35788291,
41135303,
1748982,
36249555,
42927700,
40834350,
42705552,
40855789,
782829,
36403885,
782825,
35408373,
41201043,
21141527,
21161468,
19076446,
19086411,
19129964,
40139991,
35860782,
2052764,
35859794,
19028908,
40720873,
21065487,
40058384,
40885497,
35606586,
1718277,
36266918,
43220159,
41310747,
36813154,
1592437,
21144016,
36889590,
43162537,
19123899,
36407130,
43147025,
21121803,
1748954,
41471046,
43034925,
35858564,
41470849,
40742421,
40735689,
41105139,
35862760,
43642246,
41069782,
41011267,
41435755,
43642724,
1560078,
21157850,
2050977,
1781408,
35773218,
40111243,
43522777,
36222516,
43022433,
44175180,
21104771,
1781409,
36506063,
40916364,
19102184,
43026154,
1592438,
41240889,
40908099,
41322224,
35862016,
21161283,
36881718,
21171508,
36264701,
43030474,
43167292,
21034050,
21112112,
44086142,
43169045,
41364498,
2052761,
43293562,
44113313,
36229804,
40851716,
40940294,
44094321,
41232114,
40031123,
2053054,
2052542,
44181233,
19071479,
44186410,
43206302,
36230094,
21085591,
44166257,
40851024,
44100426,
41217623,
43642889,
43155131,
40917578,
37592400,
21147040,
41070325,
43026159,
40134708,
35606591,
41279413,
41258934,
43023781,
41121903,
44164002,
36275518,
40853345,
44073039,
21112108,
35747851,
43023779,
2051328,
21092418,
35746162,
43136053,
41136961,
36074482,
40865165,
40834167,
44204946,
40867865,
43022438,
1711553,
43840343,
35862750,
43200287,
36504645,
41043832,
21033504,
21154303,
44037680,
41128203,
43140835,
19071718,
44099116,
43187312,
43217256,
36061964,
40981429,
43803831,
782443,
40947777,
21104774,
35862026,
21173500,
21112609,
21131576,
40885493,
43788521,
36062300,
43750551,
43138759,
43200290,
41231656,
41186292,
21043603,
40883375,
36218527,
42656093,
41279414,
43147256,
42927698,
21063038,
41435791,
43266281,
40948926,
43215611,
41103896,
40097502,
41287562,
43526312,
40916366,
40130101,
35862755,
43189264,
41190328,
43217257,
21072943,
43211096,
43215610,
43141606,
41434431,
36242881,
21147950,
40753358,
1787123,
21024320,
19028909,
35862769,
36062162,
36268791,
41178958,
42963130,
40960083,
41470316,
19102202,
40835548,
40855791,
41166550,
43165476,
40886177,
41435674,
41169649,
43642245,
21090949,
44041177,
21112307,
36061961,
41166552,
41058809,
41197043,
36222271,
1560076,
36233262,
21131567,
21105226,
43284817,
36248700,
44121207,
21023720,
2011842,
44168922,
41002606,
40914288,
42927759,
41094049,
40885500,
21036513,
40875587,
1725069,
40720769,
41226322,
36245932,
43138758,
44126959,
43209919,
40133800,
43037168,
46275634,
43267465,
43858118,
41073660,
40220769,
40971276,
2052664,
1510225,
42927743,
782838,
41136964,
42927702,
36218351,
35862013,
2047895
)            
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_74970258_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_74970258",
  "drug_74970258_*.csv")
message(str_glue('The data will be written to {drug_74970258_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_74970258_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_74970258_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_74970258_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_74970258_drug_df <- read_bq_export_from_workspace_bucket(drug_74970258_path)

dim(dataset_74970258_drug_df)

head(dataset_74970258_drug_df, 5)

In [ ]:
# Rename df
df_drug3 <- dataset_74970258_drug_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by drugs_query 4
dataset_74970258_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (
43136049,
21023728,
43822782,
44061821,
43767978,
2052744,
36240742,
36237142,
42683458,
21161459,
19107376,
41072410,
40720637,
41151295,
43696195,
43200292,
35764660,
1724942,
42927737,
40978757,
43026151,
43200066,
40714443,
21079262,
43022362,
36412247,
41261211,
19118483,
43026754,
21046378,
21075441,
36225760,
36214928,
2052807,
40730842,
36779745,
36237803,
36217253,
41434587,
40730829,
19124376,
43254640,
36215508,
44032815,
36241912,
40165044,
43184965,
40960832,
40051186,
36217380,
40930057,
36232859,
40720628,
1704184,
19058660,
21069578,
41042790,
42927725,
21035534,
35858551,
36225758,
1560079,
964017,
36408888,
21149330,
44187909,
21056017,
41244558,
44080039,
36403884,
40234302,
41464054,
21141524,
44060247,
43022441,
43028259,
40736234,
43167279,
41322223,
40730831,
41135307,
43171780,
35859819,
41219032,
19102203,
35859797,
21143561,
42707689,
44102438,
1710655,
42927733,
41207011,
36891950,
21023719,
2053058,
35862021,
36258815,
40971745,
35861029,
43141014,
41086186,
2052778,
44025726,
21033490,
21085597,
35862757,
21163761,
40855375,
21110078,
40720630,
41435794,
44086146,
41167764,
43026149,
43202078,
43289192,
44170994,
36219272,
42705917,
35200451,
42941337,
36258559,
36782733,
40173164,
42927746,
43147023,
42927740,
36217255,
43282591,
36275317,
41220609,
43026170,
44076523,
36061597,
43786509,
2052791,
43145156,
40746934,
43212909,
42708117,
44178719,
43624475,
1710615,
19129164,
44181183,
40917855,
36781984,
44048839,
41314692,
21033497,
19129161,
40903206,
36241907,
41123762,
36061603,
40720773,
40846848,
36218357,
37593512,
40058745,
40913946,
19038110,
42927748,
2052757,
21105229,
41105552,
43158446,
42874210,
41279412,
41022366,
43022462,
43282665,
43032445,
41435823,
35767614,
2053042,
41267144,
43146973,
19047925,
36409561,
40730833,
21171238,
40171779,
40838932,
21161286,
21043602,
43292591,
41229014,
40981434,
43022439,
41404332,
1748956,
40883372,
36062180,
43206303,
41089547,
43031556,
41054830,
40720644,
36886609,
1748921,
41226851,
42927707,
41042613,
43022420,
2031629,
43133977,
35858567,
43174019,
43136051,
1789456,
43560390,
40874081,
43714764,
43030475,
44178512,
40825714,
21144156,
19121062,
40720765,
36061909,
43154462,
36062383,
1769390,
40238934,
40730821,
43037322,
21094461,
36272735,
2052760,
41238411,
40730816,
21154417,
1718481,
43195402,
35773214,
43525935,
44166854,
2052745,
2052799,
41090167,
43517669,
41433860,
35862036,
1738170,
36271233,
43211101,
40944570,
43026139,
41047302,
36225759,
43026144,
43037166,
43184964,
42927745,
41086185,
35742103,
35862001,
40827062,
36788471,
40241984,
19103098,
42707686,
41199573,
43660213,
36061905,
41104867,
43034924,
43136095,
2011840,
21033501,
36061967,
36272379,
40832051,
41042615,
36222584,
36262839,
40238461,
2053056,
41257683,
40008581,
41107082,
36240290,
19129162,
41435810,
36061593,
36785880,
43149976,
995226,
21090324,
43161044,
43276083,
43294618,
36226567,
782837,
43271783,
43158207,
42927710,
43146974,
21033500,
36219271,
42941335,
41156726,
41163376,
19125900,
35862762,
1510223,
41042358,
19068302,
41041197,
21043601,
36276388,
2050976,
36062160,
40930181,
41163375,
41007937,
41011268,
40854228,
40917093,
2052780,
42874789,
19038784,
41062393,
42918910,
43192903,
36074448,
41470254,
43216077,
41225715,
1769389,
40949195,
21130769,
45774771,
43267464,
40891190,
35743805,
41226323,
43191019,
36062185,
36061955,
40885252,
41242639,
43026148,
35861999,
43169043,
42965766,
21024224,
21140073,
40981882,
2052773,
40733518,
40736232,
43786493,
1704213,
36882776,
19127570,
43026756,
43750002,
36248153,
41103647,
44051010,
21161458,
21026617,
2031820,
35768848,
21092419,
36214320,
21119883,
1704216,
41165665,
43269591,
43189257,
2053074,
41159232,
40989515,
40753348,
19124375,
35862009,
2052769,
40730841,
43182791,
41043833,
21171594,
41226321,
42683459,
41252484,
40724590,
21086136,
40080337,
44127515,
21070986,
1738163,
45892743,
42927708,
35862171,
43840342,
21043417,
19102238,
40948925,
2052733,
36062182,
43294620,
19109683,
43037169,
36275271,
35748013,
43162538,
2031818,
2053061,
42927705,
2053069,
36781888,
41075643,
40753346,
35862752,
35861974,
44108870,
1789429,
45775749,
964217,
41116314,
21124654,
19084908,
44204945,
2011841,
41325359,
44097430,
36273069,
41434401,
36882046,
44047869,
2053064,
2052794,
43297865,
44086145,
44158954,
43031549,
2051431,
36248152,
43212958,
40710121,
36062177,
40865477,
41323861,
21053408,
2052756,
19118481,
21105230,
35860451,
36784561,
44160496,
44110269,
21112862,
41100606,
35764515,
41293606,
35605923,
36217254,
19038651,
1724869,
40980358,
43134210,
40874079,
36238738,
41288370,
41452613,
41084695,
41136543,
41052291,
36219735,
36062171,
45775746,
40730835,
21133629,
44159467,
36240740,
2052802,
35764662,
43276253,
36219720,
43525934,
21079385,
2052762,
35756126,
40917582,
36245184,
43153882,
35412773,
41006885,
35861998,
1704185,
42683462,
36246126,
43193748,
36788472,
36275334,
1747157,
40855792,
1736924,
44047430,
41064866,
1756838,
36811452,
43184870,
21072946,
1718459,
44182671,
21164237,
43283190,
36218356,
44171490,
41029800,
40162288,
44167764,
36062301,
41259497,
36062166,
35745752,
43032455,
21142014,
36074404,
43026162,
2052759,
40220795,
43180169,
37592402,
21059451,
2052797,
40733286,
40160004,
43202022,
40724599,
42731648,
41074120,
35764665,
36239641,
36264828,
43768511,
19112911,
21082606,
964224,
40080334,
41031268,
41187847,
40853350,
793034,
44175088,
40883373,
44086144,
21177453,
21120399,
41165667,
35860453,
41094044,
19120956,
40749147,
35412586,
41288217,
41435774,
42543869,
36880110,
35862179,
36246120,
44171530,
41435734,
21134600,
41167763,
40238932,
782439,
2052790,
36213370,
2053063,
43822392,
41404462,
44183594,
2051326,
43022461,
21053216,
19038143,
36509195,
40950732,
36258898,
40051185,
35860838,
35862599,
1510227,
40753354,
35764661,
40724589,
35758650,
43026753,
36222529,
43840466,
19103097,
41304617,
43149674,
42927709,
43288723,
40882253,
36248536,
35862002,
41106651,
43195888,
44128408,
21043864,
40835038,
43215643,
43261492,
35862010,
40888606,
41118016,
41195800,
37594091,
43858446,
36784664,
43136050,
35606583,
42876172,
40917853,
40958252,
41404340,
40720640,
43026141,
40877987,
1592275,
21046018,
40746919,
2050596,
782845,
41010076,
40898934,
19089435,
41315791,
782844,
43803422,
43659805,
21151345,
21036160,
41074118,
35858566,
35862177,
40153453,
43732362,
2051666,
44048841,
44204762,
36248898,
41038515,
1711525,
41166222,
36403430,
43281761,
41469944,
36061966,
21085112,
1703095,
36403890,
35200461,
40010552,
1592276,
43294164,
35748009,
21142015,
36813059,
43195962,
36248579,
40916126,
40919108,
36062168,
2053045,
1758538,
21129588,
21128235,
21063040,
36269468,
35862765,
36898056,
36788476,
41010077,
44033550,
41296633,
40936270,
19121061,
21083115,
41073667,
35147633,
36782732,
21065978,
40746920,
40885495,
41152924,
43163816,
1736993,
44101528,
2052739,
40058381,
19113608,
43026153,
41072091,
36781983,
36222679,
44174955,
36897763,
36061614,
43166155,
21131770,
21169054,
40976101,
36223046,
1703096,
40746927,
36074497,
43178345,
19086090,
21171249,
994583,
35742563,
21024321,
2011844,
44033885,
43212905,
41040311,
40918007,
43260892,
41290899,
2052804,
40947775,
40944571,
41199150,
35862754,
964013,
41042614,
36218347,
43022443,
44086491,
40241983,
42731594,
21141525,
36220688,
40146970,
44161773,
21082601,
21143973,
36784562,
43695641,
35409175,
21030126,
41040307,
40736233,
36214476,
42874782,
42876072,
41269365,
43031555,
40919588,
2052491,
36233144,
19093751,
21131568,
40161651,
43162539,
43042972,
44073606,
43022449,
43147021,
41290904,
40161652,
43164475,
40977855,
1511231,
41156728,
41166223,
40853344,
40933489,
36239226,
41228122,
19076447,
41114498,
43195401,
36061896,
43026167,
21073418,
35860794,
40929850,
40127635,
41138064,
45892976,
42927753,
40733283,
40730822,
36229806,
44075109,
42705879,
43180171,
43144914,
41311816,
43026757,
35859796,
40919109,
43026160,
19079685,
41242638,
43163017,
41167272,
43605577,
36406313,
2052663,
43587497,
44048838,
35860793,
40724601,
41163374,
1738140,
44037116,
42874954,
19036704,
44111676,
40238457,
40860485,
44205415,
43211095,
36886489,
43522840,
21124649,
43268384,
36781985,
43272948,
44063145,
21105227,
35862176,
40844388,
43861741,
41132018,
36248899,
40980359,
19029028,
41250073,
40887120,
43261444,
2052806,
43522791,
19117195,
40884567,
40238462,
35862005,
40878424,
41011688,
37593518,
40976104,
35860784,
40032838,
40234300,
35861962,
21073419,
40947771,
40936272,
36261182,
1724944,
41123759,
36224277,
40971746,
40824389,
36223047,
42927741,
21151344,
19102212,
41011685,
782828,
42705415,
42707687,
43189259,
21043414,
44216222,
19038781,
40982927,
40720650,
21161287,
35773053,
2052753,
41103538,
44166855,
36405444,
42927760,
43282666,
43156109,
1718278,
42941331,
41041855,
36885670,
21069535,
36269541,
40978756,
44081332,
41125212,
21115052,
43163132,
41127729,
43188103,
43136052,
36247333,
36237502,
36889967,
35743810,
40945868,
19076449,
19125237,
40850925,
42708120,
41237649,
19010179,
40724597,
40072741,
35413017,
36417994,
40733281,
43202077,
1737001,
41168001,
41434033,
2052758,
43178105,
43165490,
41221489,
43560391,
21079308,
41062396,
42927757,
43037167,
44186126,
37592143,
21082832,
41123760,
40730820,
41228121,
36062304,
36788478,
40853342,
43660212,
1511091,
43526310,
43176473,
36504821,
40079018,
19022312,
44186506,
41033746,
41042365,
21124651,
40878195,
41226318,
43140950,
41435817,
43696023,
19125941,
40166034,
42941333,
21023929,
40886689,
21110077,
40917854,
43032444,
43025639,
19086111,
41288371,
43202121,
589954,
41147932,
19122307,
21120398,
36880324,
35743808,
42927719,
41258930,
21046017,
36412161,
36508732,
41323444,
36880750,
43156336,
782831,
36062186,
19010236,
36219236,
35859823,
19047810,
19084735,
41094045,
40733338,
41262847,
42709026,
44167354,
44167763,
41069786,
995222,
40753347,
41435698,
41040310,
21036482,
2052741
)            
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_74970258_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_74970258",
  "drug_74970258_*.csv")
message(str_glue('The data will be written to {drug_74970258_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_74970258_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_74970258_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_74970258_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_74970258_drug_df <- read_bq_export_from_workspace_bucket(drug_74970258_path)

dim(dataset_74970258_drug_df)

head(dataset_74970258_drug_df, 5)

In [ ]:
# Rename df
df_drug4 <- dataset_74970258_drug_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV by drugs_query 5
dataset_74970258_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (
21033491,
21023724,
43032446,
40097201,
21149332,
36062178,
44163052,
36281977,
40220793,
40886959,
44042630,
43022437,
21051769,
44063270,
35862768,
44182953,
35861971,
41069781,
2052784,
44090902,
43215612,
43169086,
43277370,
43211103,
35861973,
35773211,
36784668,
21141534,
35415359,
21144015,
40746928,
35603889,
19116993,
35862592,
40124351,
40830297,
43211098,
2053049,
42543872,
21173888,
40851565,
43160869,
36266601,
21141521,
40079019,
36244710,
19098455,
42927762,
36232126,
35862601,
44171531,
1704243,
44187337,
36061609,
1781406,
41435796,
41249680,
1724917,
43158159,
44092334,
41169650,
40746929,
1758536,
42657243,
21129667,
44170586,
42927699,
43032441,
19124378,
43178342,
35858550,
36274880,
41226852,
41433900,
36249078,
36074477,
36405840,
44180502,
19022918,
41294030,
36782725,
35862761,
40825031,
36220593,
782442,
44174865,
44167260,
35605920,
35743812,
36217344,
2052732,
36061598,
35749491,
41323445,
43037160,
43165283,
41170720,
1724827,
41230642,
40851715,
21034199,
21024498,
40968870,
1781411,
19102468,
19102273,
35415198,
21026728,
43022453,
41194553,
43767977,
41404281,
43266992,
1710658,
40947769,
35751989,
1724921,
40981435,
43696682,
35858563,
21104773,
42875453,
40095222,
40084803,
44025908,
43803832,
40866649,
43266993,
21142104,
35606587,
35858549,
19121063,
41225610,
43199152,
41038517,
43156334,
40885499,
42874222,
40238458,
35407335,
40079017,
41257155,
43208695,
21112696,
43023783,
36262483,
36230318,
1592274,
40746913,
41233031,
40950314,
36268197,
782437,
40161650,
41104622,
36788483,
40720636,
21141528,
41320316,
44028386,
43037161,
40855376,
40945869,
41011523,
35762875,
43732075,
36885141,
44041176,
35861990,
2053050,
35409381,
41133093,
19102222,
40095136,
36813248,
21052636,
40886680,
43145153,
41320140,
41411944,
36419350,
21138092,
35752144,
19120988,
41008134,
21132079,
41056869,
43217793,
21075764,
19071717,
36074480,
36781987,
19076445,
43032545,
36405838,
43154503,
41136533,
19063361,
44100456,
41318965,
19097780,
43149675,
44785498,
21053221,
41085214,
2052777,
35861995,
41470703,
43022454,
40989955,
40746931,
35748012,
43189255,
1703097,
40746910,
1748986,
40059337,
42927731,
43858972,
36225761,
40072740,
37593520,
43200285,
36062172,
36268634,
42927727,
21142106,
40746914,
43217926,
19038760,
21085488,
43140923,
40166030,
42927728,
44166835,
35409959,
35412557,
40134179,
40160001,
21159876,
2051329,
19124372,
41073827,
19084907,
21072349,
36238742,
36895806,
43220297,
40234301,
44030848,
36788493,
44161772,
44101876,
995000,
21063032,
36249554,
40238935,
43022459,
35861977,
19129168,
21115057,
41290903,
36236163,
43678368,
793030,
40236068,
40741179,
40166029,
35860792,
36248155,
43037159,
35862759,
43732580,
43037162,
36233142,
43022457,
1704215,
43176518,
21060949,
43190966,
43030472,
40981433,
42927706,
43211102,
41123758,
36896586,
41229016,
43208697,
40147470,
21066496,
41103004,
35862187,
42874786,
44052262,
41502277,
41322225,
40173162,
19112910,
41037970,
42927750,
41250071,
36781889,
21159877,
1703093,
35862014,
21134577,
36061595,
41259820,
36507464,
44185708,
42731595,
43215245,
40971262,
41362040,
43825552,
21114143,
35860781,
588226,
35862019,
43266353,
41288586,
43160871,
21134056,
36814491,
21049658,
41435792,
41226319,
42656091,
1718477,
42927747,
41168141,
41133091,
964015,
43022440,
35862467,
19125942,
35742602,
40979937,
36061973,
43522783,
43822109,
36223703,
43189253,
36509698,
41250076,
44079944,
43154501,
35775118,
42875402,
36260152,
21085595,
35862000,
43219712,
43151361,
994582,
43026150,
21056115,
44205387,
44170993,
37593519,
19048075,
35750311,
19072156,
36508857,
21071498,
37592401,
43149973,
782822,
43178341,
21124655,
35410429,
21042836,
41069785,
43733307,
36238999,
40140028,
41159151,
36239000,
21063487,
21039998,
35859798,
43043802,
21073420,
43256119,
36240741,
41135308,
40746933,
40071868,
43750001,
43167030,
40883374,
41007936,
36223045,
40730834,
41191170,
40837080,
43160867,
21104803,
43031546,
35133027,
21043419,
43022463,
43180172,
40746911,
21171051,
42927697,
44205432,
43146980,
1511084,
40710122,
44037115,
2053041,
43167293,
19068301,
40733277,
42543876,
41195631,
44062430,
44182457,
43032452,
43167281,
43176380,
1511089,
36264745,
41132577,
43732074,
40730837,
41292508,
43661061,
41094043,
21065953,
43277371,
21164236,
35859813,
782444,
40709681,
40140745,
43198335,
41229696,
19079683,
36217383,
41292115,
41073668,
41290905,
36893513,
44084514,
40166031,
41117642,
43217992,
36248701,
2053419,
21122326,
44168093,
41058810,
42705928,
782445,
36779742,
40971277,
36784666,
1736829,
36264829,
41052061,
40867261,
41123763,
21171048,
35858573,
1712889,
21121804,
40220792,
40906482,
1710312,
43256175,
41182328,
1718490,
41107081,
35861976,
19119478,
40162293,
43030469,
21154302,
41195245,
19064451,
35606584,
44096704,
42927751,
43767980,
40058385,
45892113,
40850924,
36784662,
21115081,
36788491,
40875586,
41292120,
44171443,
41155220,
36061868,
41462730,
19081172,
36788475,
35862594,
40866189,
43032447,
43160870,
42875454,
43515576,
40714442,
40084802,
21176533,
36409072,
2011836,
44176350,
41168003,
19078498,
21090948,
43522809,
43193751,
21102876,
35861994,
44176188,
40945668,
35862034,
41292121,
19086919,
35605924,
35767145,
41411969,
40134455,
42705417,
21118392,
40986222,
36062173,
40097504,
43272947,
1710314,
43032442,
35862189,
41147438,
40123652,
36219235,
43821523,
36222583,
1787122,
21142105,
44071628,
1748988,
36409074,
36241908,
21142012,
35748010,
40720279,
1718461,
41433685,
43202076,
40888607,
44174454,
42874226,
36062296,
41021877,
36062163,
40084801,
43022456,
42731574,
40976105,
41323862,
36062305,
35862004,
43031553,
21065948,
21104819,
43785856,
43277900,
21082600,
40937839,
43560388,
43212955,
782277,
1789430,
35858557,
44185672,
2052772,
43194213,
45775751,
41219031,
2011868,
40753355,
40940280,
2031630,
40720875,
41009758,
36062181,
40238933,
41003133,
41057983,
37593606,
43264140,
35862185,
40134454,
41231657,
40097503,
588779,
43026155,
44083789,
36219438,
43212908,
21121800,
43166160,
36246136,
42683460,
21132160,
40882850,
36886824,
43274241,
35770064,
35860791,
44129865,
35861993,
19102221,
42482528,
40840171,
21072942,
35858554,
43210853,
44082599,
1787101,
44061789,
41113385,
40724602,
40720627,
35860798,
2052747,
44171343,
41228123,
42927712,
35414480,
21131573,
36506008,
2052771,
36244991,
35860456,
43266352,
43022452,
44127029,
42927713,
37593515,
36226565,
21161292,
35201132,
41382003,
40733517,
41209901,
43522786,
35137520,
40238460,
41147933,
40220796,
21082607,
21144385,
44114705,
40720642,
41290900,
40171823,
19102220,
43283250,
41085213,
19070373,
36222517,
42708115,
36417995,
35789094,
44171717,
35762876,
40079021,
2052743,
41000187,
21043604,
44036985,
43202120,
36242595,
43182792,
21036029,
40981430,
21161288,
2053062,
36881853,
40906483,
36222585,
995126,
44114795,
19015458,
2052792,
40854229,
21161285,
36061965,
43642746,
40853055,
1704186,
43189260,
21161284,
36788887,
36508021,
21056018,
41117645,
43858787,
41435617,
40852102,
1727254,
2053044,
40724592,
43189258,
35862035,
35860457,
2052782,
21060950,
36230317,
43206301,
41502495,
35750946,
21085598,
40906477,
19125901,
36074450,
40961170,
40926973,
40978753,
40032843,
36061962,
41214714,
41140818,
21115056,
36222678,
35407178,
45774768,
19022955,
2050978,
36409400,
19022917,
36061956,
43279392,
36061963,
43026755,
782651,
36061957,
2052793,
41262399,
43037170,
35415357,
41239852,
35742186,
1712891,
43191064,
44177571,
21036481,
1710617,
40853349,
21045404,
1748953,
21063042,
41199157,
40886854,
35768602,
19076442,
43167291,
44182769,
43212956,
36061968,
19102237,
21061186,
43283430,
36226562,
41175939,
42683457,
41086184,
35862758,
1769391,
21029620,
40741182,
36783445,
40736235,
43804006,
44080118,
41073825,
40730817,
42656094,
35862032,
21095068,
41435750,
43032453,
36248154,
36788480,
19083609,
19129165,
2052785,
40162287,
41166549,
43031554,
40140030,
35862753,
43661593,
41430645,
43171800,
40733280,
36407767,
36880531,
40825344,
40140029,
41069221,
44052263,
42941328,
44174951,
21072944,
19086918,
43022431,
40981431,
40746923,
21055776,
41430753,
19105297,
40742422,
40947546,
42707831,
41315121,
36239642,
43022244,
43173539,
42941330,
40867864,
44169728,
36880528,
42544020,
40961371,
19010060,
40741903,
41312207,
44049938,
36788490,
35605552,
35764669,
43178346,
40071867,
1712912,
44085934,
43142642,
35788681,
2052805,
42874224,
44173306,
43263863,
43212855,
35858574,
36062306,
36784663,
40171780,
44120245,
40909012,
1781439,
43180170,
41167761,
995223,
36258999,
21100883,
19125242,
35743807,
19113858,
40058383,
40947770,
19122564,
41517302,
21026272,
43187458,
2052789,
36782731,
43188106,
21141522,
43271854,
44037806,
41292506,
2052787,
35411077,
40720645,
35858568,
44075985,
41296632,
21036483,
42927703,
36507001,
41106650,
43022450,
43190964,
21051234,
36788474,
41260972,
40949359,
36219719,
21173067,
41462582,
36273759,
43298279,
40238936,
35862003,
19064430,
40753357,
40981432,
35750312,
35604227,
36247334,
41232112,
21112315,
41074117,
35747852,
40142305,
42927738,
43167029,
44122724,
36785882,
1511083,
41195246,
21031545,
40847308,
35862600,
40943664,
21098871,
42705915,
41304618,
36404667,
43840210,
40708527,
40173163,
42543874,
36405839,
45774767,
40896672,
43642381,
19006256,
21033489,
36885205,
41209900,
589584,
41470067,
35773219,
40124350,
40724593,
35412885,
42655679,
35605555,
41031270,
43134212,
36248763,
40945665,
41292114,
36062289,
43202074,
35772957,
40720646,
19036630,
40162292,
41156727,
36061972,
41226320,
21072938,
41103895,
37594089,
19016698,
41217622,
43642244,
43269332,
21026710,
43026146,
37593311,
40886681,
36244106,
1560077,
36889901,
43786182,
41021227,
42941329,
36882944,
40097202,
36779746,
36503440,
21055612,
19076444,
40971273,
40746921,
35862006,
35754534,
35859816,
40947778,
21171509,
41273532,
35861991,
44073688,
41010079,
44111803,
37593517,
36412248,
35861969,
19113977,
36233261,
964021,
40051183,
41152357,
21131778,
21073421,
1560085,
43623982,
41100706,
43272466,
36061976,
44186505,
40976103,
35757500,
43217791,
44159451,
21104377,
43768704,
36222513,
19028907,
40877090,
40238930,
40720639,
19087771,
19116566,
36218354,
41011266,
40753345,
41226697,
41435754,
1781437,
43714174,
41041198,
40949194,
36249553,
42875457,
35862033,
43641656,
40746932,
36271211,
44159847,
37593578,
19022976,
41007447,
36277107,
44089712,
40884568,
41163926,
41060983,
43037163,
41288215,
19048071,
44065246,
40753353,
36238051,
41072411,
42874955,
41240902,
36782727,
42705553,
40882845,
41435812,
36778999,
21104820,
21063035,
40937838,
2050589,
793033,
41163924,
35415355,
43294163,
42927720,
41069784,
21073648,
44114796,
41071597,
35200463,
40977856,
21114594,
1789428,
40884569,
43749812,
43202023,
41101769,
1787125,
36232858,
43176519,
21023726,
35746783,
21150544,
40891191,
21061563,
35200467,
43288113,
1748984,
2052734,
43257505,
41226696,
36898068,
42705916,
36061602,
35606588,
19113115,
43149644,
35861992,
1727255,
40928875,
41103898,
21082598,
21101477,
36247225,
43298018,
19036706,
40720771,
964011,
1789457,
36275055,
42705878,
1704187,
36230092,
44112270,
44164619,
44067904,
41288745,
40913203,
21141529,
36406020,
21112110,
43151873,
42927723,
43026158,
40857296,
41452281,
2050979,
43212854,
43858286,
21026615,
41303338,
44037034,
36061900,
36779001,
41433352,
44032034,
36272920,
21036484,
36785884,
1756839,
44026191,
19085390,
40095223,
40753350,
36243625,
43642066,
40720872,
1511229,
36230118,
36062187,
2052752,
35858571,
44049837,
41164311,
1511087,
40937843,
41288746,
42543871,
40746924,
2052800,
35409886,
41270319,
41184181,
40905018,
40978752,
40940295,
21051768,
44160221,
35859817,
35764663,
21065837,
21053217,
2053043,
19043795,
41517301,
21141523,
36220377,
35862172,
40913803,
21138148,
21100249,
43278415,
40918005,
35760284,
35755158,
43032454,
35743806,
782440,
44182768,
43167280,
43209922,
21108646,
35862174,
40238931,
36232857,
41103897,
21056111,
21023729,
44107276,
43288722,
21177454,
44048809,
36219231,
44205026,
40992430,
44107515,
40147740,
43031559,
40917587,
19102467,
43280194,
41435600,
21114739,
41016366,
21029621,
41197947,
36220597,
41006779,
41043831,
782840,
44102713,
35756127,
41256601,
21046242,
40854873,
40720648,
43141012,
21102306,
43606601,
43191065,
40975510,
35756372,
1718476,
43144918,
40720635,
40875584,
43288111,
21153965,
21108720,
41007448,
41435801,
41256514,
42705413,
35858558,
2052781,
41115945,
36504619,
782830,
36248578,
44082746,
2052746,
21144157,
43283252,
40851566,
19103067,
21043421,
42731691,
44047340,
40162289,
40128000,
36264827,
1724919,
36783450,
36236510,
35859815,
37592139,
36061954,
43167289,
40937842,
41230643,
40051189,
41113473,
19122186,
35762310,
36267644,
1736994,
21040051,
40237541,
36779744,
41140011,
43804159,
35605551,
35858552,
2053057,
36062184,
43786181
)            
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_74970258_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_74970258",
  "drug_74970258_*.csv")
message(str_glue('The data will be written to {drug_74970258_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_74970258_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_74970258_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_74970258_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_74970258_drug_df <- read_bq_export_from_workspace_bucket(drug_74970258_path)

dim(dataset_74970258_drug_df)

head(dataset_74970258_drug_df, 5)

In [ ]:
# Rename df
df_drug5 <- dataset_74970258_drug_df

print("complete")

In [ ]:
# rbind to make 1 big drug df
hiv_drugs <- rbind(df_drug1, df_drug2, df_drug3, df_drug4, df_drug5)

print("complete")

# Process HIV by conditions, labs, and drugs

In [ ]:
# Rename dfs
hiv_condition <- dataset_38981770_condition_df
hiv_labs <- dataset_89725246_measurement_df

head(hiv_condition)
head(hiv_labs)
head(hiv_drugs)

In [ ]:
# Create column for lab test type
hiv_labs <- hiv_labs %>%
  mutate(lab_test_type = case_when(
    measurement_concept_id %in% c(
      3003852, 3006323, 3007417, 3007921, 
      3008121, 3008716, 3010021, 3011325, 
      3012733, 3013123, 3013854, 3013906, 
      3015015, 3016870, 3017675, 3018154, 
      3020171, 3020647, 3021850, 3022061, 
      3024449, 3025705, 3026601, 3027347, 
      3028734, 3030101, 3031067, 3031537, 
      3032287, 3032965, 3033479, 3034910, 
      3034979, 3035962, 3036139, 3036712, 
      3037935, 3038100, 3039289, 3039370, 
      3040430, 3042841, 3043837, 3046636, 
      3046775, 3049147, 3053246, 4136494, 
      4326419, 21492993, 21493867, 21494795, 
      36203237, 36203506, 40760007, 40761081, 
      40761994, 40771495, 42527815, 42870583, 
      43533927, 43533928, 46235448, 46272018
    ) ~ "AbAg",
    
    measurement_concept_id %in% c(
      3012693, 3029801, 3030248, 3031311, 
      3032209, 3032427, 3032441, 3032728, 
      3032957, 3039772, 3043586, 3044184, 
      3044830, 3045827, 3048236, 3050623, 
      21491517, 21491518, 21491519, 21491520, 
      21493663, 40763960, 43055394, 40760301, 
      43055393
    ) ~ "Genotype",
    
    measurement_concept_id %in% c(
      3000685, 3001240, 3005467, 3006826, 
      3010074, 3010747, 3013180, 3014347, 
      3017385, 3025907, 3026532, 3027287, 
      3028148, 3031110, 3031672, 3031839, 
      3032045, 3032527, 3032756, 3037648, 
      3038207, 3047064, 3048506, 3048528, 
      3052183, 3052357, 3052999, 4201046, 
      21492657, 40762145, 40762510, 
      40765206, 40771874, 42868715, 44816748, 
      36203199, 44790037, 3004365
    ) ~ "PCR",
    
    TRUE ~ "None"
  ))

# Summary of the new column
summary(as.factor(hiv_labs$lab_test_type))

In [ ]:
# Create column for lab result
hiv_labs <- hiv_labs %>% 
  mutate(lab_result = case_when(
    value_as_concept_name %in% c('Positive', 'Reactive', 'HIV-1 Positive', 'Repeatedly reactive', 'Preliminary positive') ~ 'Positive',
    value_as_concept_name %in% c('Negative', 'Nonreactive', 'Non-Reactive', 'Not detected', 'None detected', 'No reaction', 'Repeatedly non-reactive', 'None', 'HIV Negative', 'Normal') ~ 'Negative',
    value_as_concept_name %in% c('Indeterminate', 'Undetermined', 'Inconclusive') ~ 'Indeterminate',
    is.na(value_as_concept_name) ~ 'Unknown',
    value_as_concept_name %in% c('No matching concept', 'Unknown', 'Abnormal', 'Invalid', 'Detected') ~ 'Unknown',
    value_as_concept_name %in% c('Not performed') ~ 'Not performed',
    lab_test_type %in% c('AbAg') & value_as_concept_name %in% c('=') ~ 'Unknown',
    TRUE ~ 'Unknown'
  ))

# Summary of the new column
summary(as.factor(hiv_labs$lab_result))

In [ ]:
# Remove VL concept measurements since already accounted for in VL phenotyping
hiv_labs2 <- hiv_labs %>% filter(!measurement_concept_id %in% c(3010747,
3013180,
3031527,
3032527,
3031839,
3048506,
40765206,
3010074,
3048528,
40762510,
3026532,
3031382,
3028148,
3052999,
3039421,
42868715,
3048535,
4201046,
3038207, 3032756, 3017385, 44792202, 3014347, 3031672, 3031110, 4163881, 4040194, 3000685, 3006826, 3025907, 3052183, 44816748, 40762145, 40771874, 21492657, 3047064, 3052357, 3027287, 3032045, 3001240, 3005467, 3037648, 36203199, 44790037, 3004365))

nrow(hiv_labs)
nrow(hiv_labs2)

In [ ]:
# Get list of person ids from conditions, labs, and drugs
# Conditions
hiv_condition_person_id <- hiv_condition %>% distinct(person_id)

# Identify individuals with positive hiv lab test
hiv_labs_person_id <- hiv_labs2 %>% filter(lab_result == 'Positive' | lab_test_type == 'Genotype') %>% 
                        distinct(person_id)

# Drugs
hiv_drug_person_id <- hiv_drugs %>% distinct(person_id)

nrow(hiv_condition_person_id)
nrow(hiv_labs_person_id)
nrow(hiv_drug_person_id)

In [ ]:
# Create list of unique person ids from each hiv cohort

# rbind
hiv_c_l_d <- rbind(hiv_condition_person_id, hiv_labs_person_id, hiv_drug_person_id)

# Filter on distinct individuals
hiv_c_l_d2 <- hiv_c_l_d %>% distinct(person_id)

head(hiv_c_l_d2)
nrow(hiv_c_l_d2)

# Process CD4 and VL data

In [ ]:
library(tidyverse)
library(bigrquery)

# HIV VL measurements
dataset_89725246_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
3013180,
3039421,
3048506,
3031839,
3052999,
40762510,
3032527,
3048528,
3028148,
3010074,
3010747,
42868715,
40765206,
3031382,
3031527,
4201046,
3026532,
3048535,
3038207, 
3032756, 
3017385, 
44792202, 
3014347, 
3031672, 
3031110, 
4163881, 
4040194, 
3000685, 
3006826, 
3025907, 
3052183, 
44816748, 
40762145, 
40771874, 
21492657, 
3047064, 
3052357, 
3027287, 
3032045, 
3001240, 
3005467, 
3037648, 
36203199, 
44790037, 
3004365
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_89725246_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_89725246",
  "measurement_89725246_*.csv")
message(str_glue('The data will be written to {measurement_89725246_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89725246_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_89725246_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_89725246_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89725246_measurement_df <- read_bq_export_from_workspace_bucket(measurement_89725246_path)

dim(dataset_89725246_measurement_df)

head(dataset_89725246_measurement_df, 5)

In [ ]:
# Rename df
df_vl <- dataset_89725246_measurement_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# CD4 percent measurements
dataset_89725246_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
3014037
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_89725246_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_89725246",
  "measurement_89725246_*.csv")
message(str_glue('The data will be written to {measurement_89725246_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89725246_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_89725246_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_89725246_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89725246_measurement_df <- read_bq_export_from_workspace_bucket(measurement_89725246_path)

dim(dataset_89725246_measurement_df)

head(dataset_89725246_measurement_df, 5)

In [ ]:
# Rename df
df_cd4_percent <- dataset_89725246_measurement_df

print("complete")

In [ ]:
library(tidyverse)
library(bigrquery)

# CD4 absolute measurements
dataset_89725246_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
2212814,
3028167,
40768447
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_89725246_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_89725246",
  "measurement_89725246_*.csv")
message(str_glue('The data will be written to {measurement_89725246_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_89725246_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_89725246_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_89725246_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_89725246_measurement_df <- read_bq_export_from_workspace_bucket(measurement_89725246_path)

dim(dataset_89725246_measurement_df)

head(dataset_89725246_measurement_df, 5)

In [ ]:
# Rename df
df_cd4_absolute <- dataset_89725246_measurement_df

print("complete")

## Process VL data

In [ ]:
# Make column for integer or not in VL data
df_vl <- df_vl %>% 
  mutate(value_as_number_integer_status = ifelse(
    value_as_number == floor(value_as_number), "Integer", "Not integer"
  ))


head(df_vl[,c("value_as_number", "value_as_number_integer_status")])
head(df_vl$value_as_number_integer_status)

In [ ]:
# Look at data
head(df_vl)
colnames(df_vl)
summary(df_vl$value_as_number)

In [ ]:
# Create column for VL group
df_vl <- df_vl %>% mutate(vl_group = case_when(
    value_as_number >= 50 & value_as_number_integer_status == "Integer" ~ "VL_gr50",
    value_as_number < 50 & value_as_number_integer_status == "Integer" ~ "VL_less50",
    value_as_number >= 1.698970004 & value_as_number_integer_status == "Not integer" ~ "VL_gr50",
    value_as_number < 1.698970004 & value_as_number_integer_status == "Not integer" ~ "VL_less50",
    value_as_concept_name %in% c('Positive','Abnormal', 'Reactive', 'Detected') ~ "VL_unknown",
    value_as_concept_name %in% c('High', '87') ~ "VL_gr50",
    value_source_value %in% c('<20', '<20 NOT DETE', '<20 DETECTED', '<20 NOT DETECTED', '<1.30 NOT DE' , '<1.30 DETECT', '<1.30 DETECTED', '<1.30 NOT DETECTED', '<1.3', '<1.30', 'DETECTED, <20', '<40', '<1.47') ~ "VL_less50",
    value_as_concept_name %in% c('Negative', 'Normal', 'Nonreactive', 'Non-reactive', 'Non-Reactive', 'Not detected', 'Low', 'Not detected/negative', 'DNR', 'Stable') ~ "VL_less50",
    is.na(value_as_number) & is.na(value_as_concept_name) | value_as_concept_name %in% c("No matching concept", "Indeterminate", "Uncertain", "Equivocal", "Not provided", "Not tested", "Null", "Refused", "Report", "Service comment", "Test not performed") ~ "VL_unknown",
    TRUE ~ "None"
))

summary(as.factor(df_vl$vl_group))                          

In [ ]:
# QC looking at 20 rows of each group
head(df_vl %>% filter(vl_group == "None") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, vl_group),20)

summary(df_vl %>% filter(vl_group == "None") %>% pull(value_as_concept_name) %>% as.factor())


## Process CD4 data

In [ ]:
head(df_cd4_absolute)

In [ ]:
# Create column for integer status
df_cd4_absolute <- df_cd4_absolute %>% 
  mutate(value_as_number_integer_status = ifelse(
    value_as_number == floor(value_as_number), "Integer", "Not integer"
  ))

head(df_cd4_absolute)

In [ ]:
# Get summary of units
summary(as.factor(df_cd4_absolute$unit_source_value))
summary(as.factor(df_cd4_absolute$value_as_concept_name))

In [ ]:
# Create column for CD4 group - Updated to handle numeric values in value_as_concept_name
df_cd4_absolute <- df_cd4_absolute %>% mutate(
  # Extract numeric values from value_as_concept_name
  numeric_concept_value = case_when(
    # If value_as_concept_name can be converted to numeric, do so
    !is.na(suppressWarnings(as.numeric(value_as_concept_name))) ~ as.numeric(value_as_concept_name),
    TRUE ~ NA_real_
  ),
  
  # Create CD4 group using both standard and extracted numeric values
  cd4_group = case_when(
    
    # Unknown - keeping your original conditions
    unit_source_value == "{ratio}" |
    is.na(value_as_number) & is.na(numeric_concept_value) & 
    value_as_concept_name %in% c("No matching concept", "Indeterminate", "Service comment", 
                               "Test not performed", "Not tested", "Final", "Refused", "Null") ~ "Unknown",
      
    # CD4 > 200 from standard column or concept name
    (unit_source_value == "%" & value_as_number > 14) |
    (unit_source_value %in% c("10*3", "10*3/uL", "10*3/mm3", "10*9/L") & value_as_number > 0.2) | 
    (!is.na(value_as_number) & value_as_number > 200) | 
    (!is.na(numeric_concept_value) & numeric_concept_value > 200) |
    value_as_concept_name == "Normal" ~ "CD4_gr200",
    
    # CD4 <= 200 from standard column or concept name
    (unit_source_value == "%" & value_as_number <= 14) |
    (unit_source_value %in% c("10*3", "10*3/uL", "10*3/mm3", "10*9/L") & value_as_number > 0.2) | 
    (!is.na(value_as_number) & value_as_number <= 200) | 
    (!is.na(numeric_concept_value) & numeric_concept_value <= 200) | 
    value_as_concept_name == "Negative" ~ "CD4_less200",
    
    # Default case
    TRUE ~ "Neither"
  )
)

### QC checks

In [ ]:
# QC to check when unit is percent
head(df_cd4_absolute %>% filter(unit_source_value == "%") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group))

In [ ]:
# QC to check on neither group
head(df_cd4_absolute %>% filter(cd4_group == "Neither") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group),20)


In [ ]:
# Check count in CD4 gr, less, and unknown groups
summary(as.factor(df_cd4_absolute$cd4_group))

In [ ]:
# QC to check on CD4 gr200 group
head(df_cd4_absolute %>% filter(cd4_group == "CD4_gr200") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group),20)
summary(df_cd4_absolute$value_as_number[df_cd4_absolute$cd4_group == "CD4_gr200"])

In [ ]:
# QC to check on value of 10000000
head(df_cd4_absolute %>% arrange(desc(value_as_number)) %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group))

*I think this is fine*

In [ ]:
# QC to check on CD4 less200 group
head(df_cd4_absolute %>% filter(cd4_group == "CD4_less200") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group),20)
summary(df_cd4_absolute$value_as_number[df_cd4_absolute$cd4_group == "CD4_less200"])

In [ ]:
# Count unique person IDs with CD4_less200 and ratio units
unique_persons_count <- df_cd4_absolute %>% 
  filter(cd4_group == "Unknown" & unit_source_value == "{ratio}") %>% 
  summarise(unique_persons = n_distinct(person_id)) %>%
  pull(unique_persons)

# Print the count
print(paste0("Number of unique persons with CD4_less200 and ratio units: ", unique_persons_count))

# Show sample of these records
head(df_cd4_absolute %>% 
  filter(cd4_group == "Unknown" & unit_source_value == "{ratio}") %>% 
  select(person_id, standard_concept_name, value_as_number, value_as_number_integer_status, 
         value_as_concept_name, value_source_value, unit_source_value, cd4_group), 20)

In [ ]:
# QC
head(df_cd4_absolute %>% filter(cd4_group == "Unknown") %>% select(standard_concept_name, value_as_number, value_as_number_integer_status, value_as_concept_name, value_source_value, unit_source_value, cd4_group),20)


# Process PrEP data

In [ ]:
# Identify potential HIV individuals only on prep drugs

# Define the prep concept IDs
prep_concept_ids <- c(43026166,
36921138,
35604225,
35412092,
36225757,
782823,
36959732,
40746912,
36882760,
43026171,
36239227,
35606589,
35606581,
782826,
40746917,
36932261,
42656090,
1971581,
40917580,
44067769,
36249552,
40948924,
35411824,
782827,
1831221,
994687,
36275044,
36257199,
43026168,
36937696,
36944985,
994686,
35410515,
41211425,
2918772,
43026161,
41042359,
35897516,
40746915,
40917581,
1831210,
43202122,
41086183,
35606585,
43026163,
35606582,
35411296,
35604228,
41064865,
41260968,
782832,
36932750,
40746918,
40971261,
35604229,
1971578,
43026164,
36944859,
36811149,
40917579,
35886542,
1971580,
782824,
35411176,
35606590,
43158210,
40867643,
35407590,
782833,
36967446,
36812938,
40746916,
43026165,
40746922,
1710316,
42656092,
1758829,
43026169,
21094460,
35604224,
782834,
41136532,
41221487,
2918777,
35411297,
41260969,
36273267,
44037677,
1758830,
35411467,
36812153,
2918780,
2918773,
36249555,
782829,
782825,
35408373,
35606586,
43147025,
36951188,
44175180,
36881718,
40917578,
37592400,
35606591,
42656093,
40948926,
36939989,
2918775,
41073660,
36936730,
21035534,
21143561,
40855375,
43026170,
44076523,
44181183,
36062383,
36036105,
21094461,
41086185,
41042358,
40948925,
1971583,
43212958,
44110269,
21133629,
1971579,
41064866,
44182671,
43026162,
37592402,
35412586,
44128408,
35606583,
40746919,
36813059,
1831220,
40746920,
44086491,
35409175,
36239226,
43026167,
43026160,
782828,
2918776,
43202121,
782831,
1831222,
1758831,
36964352,
43169086,
40124351,
36926178,
35606587,
35407335,
40746913,
36268197,
35897517,
36813248,
40886680,
41136533,
19063361,
40746910,
40746914,
36249554,
1971585,
1703093,
1831209,
2918774,
21114143,
36814491,
42656091,
40979937,
36509698,
36948478,
37592401,
782822,
40746911,
41292115,
2053419,
36960014,
44176350,
1831211,
40940280,
1971582,
36926994,
21045404,
41086184,
40909012,
41167761,
2918779,
21173067,
35604227,
40124350,
41292114,
2918778,
36503440,
40746921,
35886543,
21104377,
36249553,
36271211,
1971584,
40992430,
43191065,
782830)

# Step 1: Get all person IDs who have any drug concept
all_persons_drugs <- hiv_drugs %>%
  group_by(person_id) %>%
  summarize(
    has_prep = any(drug_concept_id %in% prep_concept_ids),
    has_other_drugs = any(!(drug_concept_id %in% prep_concept_ids))
  )

# Step 2: Filter for persons who ONLY have ritonavir (has ritonavir but no other drugs)
prep_only_persons <- all_persons_drugs %>%
  filter(has_prep == TRUE & has_other_drugs == FALSE) %>%
  select(person_id)

# Step 3: Filter the original dataset to only include these persons
prep <- hiv_drugs %>%
  inner_join(prep_only_persons, by = "person_id")

head(prep)

In [ ]:
# Filter on distinct person id and drug concept name
prep2 <- prep %>% distinct(person_id, standard_concept_name)

head(prep2)

In [ ]:
# Create cohort of individuals identified by conditions and labs
hiv_c_l <- rbind(hiv_condition %>% distinct(person_id), hiv_labs_person_id %>% distinct(person_id))

hiv_c_l2 <- hiv_c_l %>% distinct(person_id)

head(hiv_c_l2)
nrow(hiv_c_l2)

In [ ]:
# Filter on distinct person id not in hiv by conditions or labs
prep3 <- prep2 %>% filter(!person_id %in% hiv_c_l2$person_id)

head(prep3)

In [ ]:
## Identify individuals in prep cohort with HBV dx
## These individuals take tenofovir so don't want to misclassify


In [ ]:
library(tidyverse)
library(bigrquery)

# HBV by conditions
dataset_38981770_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (
197795, 197493, 192240, 439674, 4281232
)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_38981770_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_38981770",
  "condition_38981770_*.csv")
message(str_glue('The data will be written to {condition_38981770_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_38981770_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_38981770_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_38981770_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), condition_type_concept_name = col_character(), stop_reason = col_character(), visit_occurrence_concept_name = col_character(), condition_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), condition_status_source_value = col_character(), condition_status_concept_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_38981770_condition_df <- read_bq_export_from_workspace_bucket(condition_38981770_path)

dim(dataset_38981770_condition_df)

head(dataset_38981770_condition_df, 5)

In [ ]:
# Rename df
hbv <- dataset_38981770_condition_df

head(hbv)

In [ ]:
# Remove individuals with HBV from prep cohort
prep4 <- prep3 %>% filter(!person_id %in% hbv$person_id)

length(unique(prep4$person_id))
length(unique(prep3$person_id))

In [ ]:
# Filter on individuals in PrEP cohort without positive CD4 or VL measurements
prep5 <- prep4 %>% 
  filter(!(person_id %in% (df_vl %>% filter(vl_group == "VL_gr50") %>% pull(person_id)) | 
           person_id %in% (df_cd4_absolute %>% filter(cd4_group == "CD4_less200") %>% pull(person_id))))

length(unique(prep5$person_id))
length(unique(prep4$person_id))

In [ ]:
# Filter on just prep person ids
prep6 <- prep5 %>% distinct(person_id)

head(prep6)
nrow(prep6)

In [ ]:
# Get hiv details with prep patients

# HIV conditions
prep_hiv_conditions <- merge(prep6, hiv_condition, by = "person_id")

# VL measurements
prep_hiv_vl <- merge(prep6, df_vl, by = "person_id")

# HIV lab measurements
prep_hiv_labs <- merge(prep6, hiv_labs, by = "person_id")

# CD4 measurements
prep_hiv_cd4 <- merge(prep6, df_cd4_absolute, by = "person_id")

# HIV drugs
prep_hiv_drugs <- merge(prep6, hiv_drugs, by = "person_id")

head(prep_hiv_conditions)
head(prep_hiv_vl)
head(prep_hiv_labs)
head(prep_hiv_cd4)
head(prep_hiv_drugs)

In [ ]:
# Make master table of hiv details in prep patients

# prep hiv conditions
prep_hiv_conditions2 <- prep_hiv_conditions %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(condition_start_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = condition_source_value,
    final_value_as_number = NA_real_,
    final_unit_source_value = NA_character_,
    final_operator_concept_id = NA_integer_,
    final_value_as_concept_name = NA_character_,
    final_unit_concept_name = NA_character_,
    final_value_source_value = NA_character_,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    final_lab_test_type = NA_character_,
    final_lab_result = NA_character_,
    indicator = "condition"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    final_lab_test_type,
    final_lab_result,
    indicator
  )

# prep hiv lab measurements
prep_hiv_labs2 <- prep_hiv_labs %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    final_lab_test_type = lab_test_type,
    final_lab_result = lab_result,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    final_lab_test_type,
    final_lab_result,
    indicator
  )


# prep vl measurements
prep_hiv_vl2 <- prep_hiv_vl %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = vl_group,
    final_cd4_group = NA_character_,
    final_lab_test_type = "VL",
    final_lab_result = NA_character_,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    final_lab_test_type,
    final_lab_result,
    indicator
  )

# prep cd4 measurements
prep_hiv_cd42 <- prep_hiv_cd4 %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = NA_character_,
    final_cd4_group = cd4_group,
    final_lab_test_type = "CD4",
    final_lab_result = NA_character_,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    final_lab_test_type,
    final_lab_result,
    indicator
  )

# prep hiv drugs
prep_hiv_drugs2 <- prep_hiv_drugs %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(drug_exposure_start_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = NA_real_,
    final_unit_source_value = NA_character_,
    final_operator_concept_id = NA_integer_,
    final_value_as_concept_name = NA_character_,
    final_unit_concept_name = NA_character_,
    final_value_source_value = NA_character_,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    final_lab_test_type = NA_character_,
    final_lab_result = NA_character_,
    indicator = "drug"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    final_lab_test_type,
    final_lab_result,
    indicator
  )


head(prep_hiv_conditions2)
head(prep_hiv_labs2)
head(prep_hiv_vl2)
head(prep_hiv_cd42)
head(prep_hiv_drugs2)

In [ ]:
# Merge into 1 df
prep_combine <- rbind(prep_hiv_conditions2,prep_hiv_vl2,prep_hiv_labs2,prep_hiv_cd42,prep_hiv_drugs2)

head(prep_combine)

In [ ]:
# Add column combining lab result, CD4 result, and VL result
prep_combine <- prep_combine %>% 
  mutate(final_lab_result2 = case_when(
    !(indicator %in% c("lab")) ~ "No lab",
    !is.na(final_lab_result) ~ final_lab_result,
    is.na(final_lab_result) & final_vl_group == "VL_less50" ~ "Negative",
    is.na(final_lab_result) & final_cd4_group == "CD4_gr200" ~ "Negative",
    is.na(final_lab_result) & final_vl_group == "VL_gr50" ~ "Positive",
    is.na(final_lab_result) & final_cd4_group == "CD4_less200" ~ "Positive",
    TRUE ~ "Unknown"
  ))

# Summary of the new column (also corrected this line)
summary(as.factor(prep_combine$final_lab_result2))

In [ ]:
# Group by person id and concatenate distinct indicator values with " and "
prep_combine2 <- prep_combine %>%
  group_by(person_id) %>%
  summarise(indicator2 = paste0(unique(indicator), collapse = " and "))

head(prep_combine2)

In [ ]:
# Get list of individuals with negative lab result
prep_lab_negative <- prep_combine %>% filter(final_lab_result2 == "Negative") %>%
    distinct(person_id, final_lab_result2)

head(prep_lab_negative)

In [ ]:
# Merge lab results
prep_combine3 <- merge(prep_combine2, prep_lab_negative, by = "person_id", all.x = TRUE)

head(prep_combine3)

In [ ]:
# Make column for confidence level PrEP phenotyping
    # level 1 (highest confidence): labs with negative test + anything else
    # level 2: drugs + labs but no lab results
    # level 3 (lowest confidence): only drugs with no lab results

# Add column combining lab result, CD4 result, and VL result
prep_combine3 <- prep_combine3 %>% 
  mutate(confidence_level = case_when(
    final_lab_result2 == "Negative" ~ 1,
      indicator2 %in% c("drug and lab", "lab and drug") & is.na(final_lab_result2) ~ 2,
      indicator2 == "drug" ~ 3,
    TRUE ~ 0
  ))

head(prep_combine3)
# Summary of the new column (also corrected this line)
summary(as.factor(prep_combine3$confidence_level))

# Continue HIV phenotyping

In [ ]:
# Exclude individuals in PrEP cohort
hiv_cld3 <- hiv_c_l_d2 %>% filter(!person_id %in% prep_combine3$person_id) %>% distinct(person_id)

head(hiv_cld3)
length(unique(hiv_cld3$person_id))
length(unique(hiv_c_l_d2$person_id))

In [ ]:
# Get hiv details with prep patients

# HIV conditions
hiv_hiv_conditions <- merge(hiv_cld3, hiv_condition, by = "person_id")

# VL measurements
hiv_hiv_vl <- merge(hiv_cld3, df_vl, by = "person_id")

# HIV lab measurements
hiv_hiv_labs <- merge(hiv_cld3, hiv_labs, by = "person_id")

# CD4 measurements
hiv_hiv_cd4 <- merge(hiv_cld3, df_cd4_absolute, by = "person_id")

# HIV drugs
hiv_hiv_drugs <- merge(hiv_cld3, hiv_drugs, by = "person_id")

head(hiv_hiv_conditions)
head(hiv_hiv_vl)
head(hiv_hiv_labs)
head(hiv_hiv_cd4)
head(hiv_hiv_drugs)

In [ ]:
# Make master table of hiv details in hiv patients

# prep hiv conditions
hiv_hiv_conditions2 <- hiv_hiv_conditions %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(condition_start_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = condition_source_value,
    final_value_as_number = NA_real_,
    final_unit_source_value = NA_character_,
    final_operator_concept_id = NA_integer_,
    final_value_as_concept_name = NA_character_,
    final_unit_concept_name = NA_character_,
    final_value_source_value = NA_character_,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    hiv_lab_test_type = NA_character_,
    hiv_lab_result = NA_character_,
    indicator = "condition"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    hiv_lab_test_type,
    hiv_lab_result,
    indicator
  )

# prep hiv lab measurements
hiv_hiv_labs2 <- hiv_hiv_labs %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    hiv_lab_test_type = lab_test_type,
    hiv_lab_result = lab_result,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    hiv_lab_test_type,
    hiv_lab_result,
    indicator
  )


# prep vl measurements
hiv_hiv_vl2 <- hiv_hiv_vl %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = vl_group,
    final_cd4_group = NA_character_,
    hiv_lab_test_type = "VL",
    hiv_lab_result = NA_character_,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    hiv_lab_test_type,
    hiv_lab_result,
    indicator
  )

# prep cd4 measurements
hiv_hiv_cd42 <- hiv_hiv_cd4 %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(measurement_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = value_as_number,
    final_unit_source_value = unit_source_value,
    final_operator_concept_id = operator_concept_id,
    final_value_as_concept_name = value_as_concept_name,
    final_unit_concept_name = unit_concept_name,
    final_value_source_value = value_source_value,
    final_vl_group = NA_character_,
    final_cd4_group = cd4_group,
    hiv_lab_test_type = "CD4",
    hiv_lab_result = NA_character_,
    indicator = "lab"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    hiv_lab_test_type,
    hiv_lab_result,
    indicator
  )

# prep hiv drugs
hiv_hiv_drugs2 <- hiv_hiv_drugs %>% 
  mutate(
    condition_measurement_or_drug_date = as.Date(drug_exposure_start_datetime),
    condition_measurement_or_drug_name = standard_concept_name,
    condition_source_value = NA_character_,
    final_value_as_number = NA_real_,
    final_unit_source_value = NA_character_,
    final_operator_concept_id = NA_integer_,
    final_value_as_concept_name = NA_character_,
    final_unit_concept_name = NA_character_,
    final_value_source_value = NA_character_,
    final_vl_group = NA_character_,
    final_cd4_group = NA_character_,
    hiv_lab_test_type = NA_character_,
    hiv_lab_result = NA_character_,
    indicator = "drug"
  ) %>% 
  distinct(
    person_id, 
    condition_measurement_or_drug_date, 
    condition_measurement_or_drug_name, 
    condition_source_value,
    final_value_as_number,
    final_unit_source_value,
    final_operator_concept_id,
    final_value_as_concept_name,
    final_unit_concept_name,
    final_value_source_value,
    final_vl_group,
    final_cd4_group,
    hiv_lab_test_type,
    hiv_lab_result,
    indicator
  )


head(hiv_hiv_conditions2)
head(hiv_hiv_labs2)
head(hiv_hiv_vl2)
head(hiv_hiv_cd42)
head(hiv_hiv_drugs2)

In [ ]:
# Make master table of HIV cohort with HIV details
hiv_combine <- rbind(hiv_hiv_conditions2,hiv_hiv_labs2,hiv_hiv_vl2,hiv_hiv_cd42,hiv_hiv_drugs2)

head(hiv_combine)
length(unique(hiv_combine$person_id))

In [ ]:
# Make column for lab results
hiv_combine <- hiv_combine %>% 
  mutate(lab_result = case_when(
    !is.na(hiv_lab_result) ~ 0,
    TRUE ~ 1
  ))

head(hiv_combine)

In [ ]:
# Make column for indicator2 (how individual was added to HIV cohort)
hiv_combine2 <- hiv_combine %>%
  group_by(person_id) %>%
  summarise(indicator2 = paste0(unique(indicator), collapse = " and "))

head(hiv_combine2)

In [ ]:
# Merge to get indicator2 column
hiv_combine3 <- merge(hiv_combine, hiv_combine2, by = "person_id")

head(hiv_combine3)

In [ ]:
# For each person id, record if they ever have the following criteria
#### 1) any VL measurement greater than 50
#### 2) count the number of VL measurements
#### 3) count the number of days between first and last VL measurements
#### 4) count the number of CD4 measurements
#### 5) count the number of days between first and last CD4 measurements
#### 6) ever have no lab result lab_result recorded
#### 7) ever have a HIV genotype recorded
#### 8) ever have a positive HIV screening Ab/Ag test positive
#### 9) ever detected as HIV patient by condition
#### 10) count the number of condition records
#### 11) count the number of days between first and last condition diagnosis
#### 12) count the number of any HIV lab records
#### 13) count the number of days between first and last any HIV lab records
#### 14) ever detected as HIV patient by drug
#### 15) count the number of drug records
#### 16) count the number of days between first and last drug exposure

# Make columns for later
hiv_combine4 <- hiv_combine3 %>%
  group_by(person_id) %>%
  summarize(
    # 1) Binary flag for viral load > 50
    vl_gr50_binary = max(case_when(
      final_vl_group %in% c("VL_gr50") ~ 1,
      TRUE ~ 0
    )),
    
    # 2) Count of VL measurements
    vl_count_measurements = sum(case_when(
      final_vl_group %in% c("VL_gr50", "VL_less50", "VL_unknown") ~ 1,
      TRUE ~ 0
    )),
    
    # 3) Days between first and last VL measurement
    vl_days_between_first_last_measurement = as.numeric(
      max(case_when(
        !is.na(final_vl_group) ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE) -
      min(case_when(
        !is.na(final_vl_group) ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE)
    ),
    
    # 4) Count of CD4 measurements
    cd4_count_measurements = sum(case_when(
      final_cd4_group %in% c("CD4_gr200", "CD4_less200") ~ 1,
      TRUE ~ 0
    )),
    
    # 5) Days between first and last CD4 measurement
    cd4_days_between_first_last_measurement = as.numeric(
      max(case_when(
        !is.na(final_cd4_group) ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE) -
      min(case_when(
        !is.na(final_cd4_group) ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE)
    ),
    
    # 6) Lab result binary
    lab_result2 = max(case_when(
      lab_result %in% c("no_lab_result") ~ 0,
      TRUE ~ 1
    )),
    
    # 7) HIV genotype binary
    hiv_genotype_binary = max(case_when(
      hiv_lab_test_type %in% c("Genotype") ~ 1,
      TRUE ~ 0
    )),
    
    # 8) HIV positive screen binary
    hiv_screen_positive_binary = max(case_when(
      hiv_lab_test_type %in% c("AbAg") & lab_result %in% c("Positive") ~ 1,
      TRUE ~ 0
    )),
    
    # 9) HIV condition binary
    hiv_pt_condition_binary = max(case_when(
      indicator %in% c("condition") ~ 1,
      TRUE ~ 0
    )),
    
    # 10) Condition count
    condition_count = sum(case_when(
      indicator %in% c("condition") ~ 1,
      TRUE ~ 0
    )),
    
    # 11) Days between first and last diagnosis
    condition_days_between_first_last_dx = as.numeric(
      max(case_when(
        indicator %in% c("condition") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE) -
      min(case_when(
        indicator %in% c("condition") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE)
    ),
    
    # 12) HIV lab count
    any_hiv_lab_count = sum(case_when(
      indicator %in% c("lab") ~ 1,
      TRUE ~ 0
    )),
    
    # 13) Days between first and last any HIV lab
    days_between_first_last_any_hiv_lab = as.numeric(
      max(case_when(
        indicator %in% c("lab") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE) -
      min(case_when(
        indicator %in% c("lab") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE)
    ),
    
    # 14) HIV drug binary
    hiv_pt_drug_binary = max(case_when(
      indicator %in% c("drug") ~ 1,
      TRUE ~ 0
    )),
    
    # 15) Drug count
    drug_count = sum(case_when(
      indicator %in% c("drug") ~ 1,
      TRUE ~ 0
    )),
    
    # 16) Days between first and last drug exposure
    days_between_first_last_drug_exp = as.numeric(
      max(case_when(
        indicator %in% c("drug") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE) -
      min(case_when(
        indicator %in% c("drug") ~ condition_measurement_or_drug_date,
        TRUE ~ as.Date(NA)
      ), na.rm = TRUE)
    )
  )

In [ ]:
# Look at data
head(hiv_combine4)
length(unique(hiv_combine4$person_id))

In [ ]:
# QC checking a couple of indivdiuals
head(hiv_combine3 %>% filter(person_id == "1000344") %>% arrange(condition_measurement_or_drug_date))
head(hiv_combine3 %>% filter(person_id == "1000396") %>% arrange(condition_measurement_or_drug_date))

In [ ]:
# Merge to get indicator2
hiv_combine5 <- merge(hiv_combine4, hiv_combine2, by = "person_id")

head(hiv_combine5)

## Set up PEP cohort

In [ ]:
# Filter on individuals in HIV cohort by drugs only
pep1 <- hiv_combine2 %>% filter(indicator2 == "drug")

head(pep1)

In [ ]:
# Merge to get all HIV drug records
pep2 <- merge(pep1, hiv_hiv_drugs, by = "person_id")

head(pep2)

In [ ]:
colnames(pep2)

In [ ]:
# Make columns for whether drug is truvada or PEP integrase inhibitor
pep2 <- pep2 %>% 
  mutate(
    truvada_binary = ifelse(drug_concept_id %in% c(21094460, 43202121, 36503440, 994686, 43147025, 36225757, 40124350, 36509698, 994687, 36814491, 40124351, 36225758, 36882760, 782823, 782822, 21114143, 35606582, 35606581, 35606583, 35606586, 35606585, 35606587, 35606590, 35606589, 35606591, 782825, 782824, 21143561, 782831, 782832, 782833, 782826, 782827, 782828, 782829, 782830, 21173067, 21035534, 21133629, 21104377, 36812938, 36881718, 43212958, 43191065, 43202122, 43158210, 43169086, 43026164, 43026163, 43026166, 43026161, 43026165, 43026162, 43026160, 36257199, 36813059, 36812153, 36813248, 36811149, 36268197, 36273267, 36271211, 40746910, 2053419, 36275044, 40746915, 40746911, 43026169, 40746914, 43026168, 43026171, 40746913, 43026167, 43026170, 40746912, 40746916, 40746921, 40746917, 40746920, 40746919, 36062383, 40746918, 40746922, 41064865, 41042358, 40948925, 40917580, 41260969, 41260968, 41292114, 40917579, 40917578, 41136533, 40948926, 40855375, 40886680, 41136532, 40948924, 41221487, 41042359, 44182671, 41167761, 44076523, 40992430, 40867643, 19063361, 1703093, 1710316, 36239226, 36239227), 1, 0),
    PEP_integrase_inhibitor_binary = ifelse(drug_concept_id %in% c(36249782, 36249784, 40720870, 40720869, 40720871, 40720872, 36249786, 36249788, 40720874, 40720873, 40720875, 40720876, 43560387, 43560388, 21119883, 44160177, 40886854, 41323598, 44171490, 41073827, 21031545, 40878195, 41073825, 37593578, 21051234, 21149422, 41273279, 43560390, 43560391, 36244925, 1718279, 36218527, 36218528, 1712889, 42705413, 42705415, 42707686, 42707687, 21065953, 21036483, 44036361, 21036482, 44785498, 44785499, 35409959, 35411785, 40720279, 40720278, 40720280, 40720281, 40720768, 40720766, 40720283, 40720282, 40720765, 40720767, 44161036, 44161773, 44161772, 44187337, 21105227, 21134574, 21105228, 21124656, 21154397, 21046356, 36408888, 36409073, 36409072, 36405444, 36407765, 36406644, 42705417, 42705418, 42707688, 42707689, 21115057, 21124655, 44101098, 21085595, 36405840, 36404479, 36409074, 36407004, 36404667, 36406645, 19127570, 19020192, 43138382, 43149294, 41121903, 44163224, 43215247, 44159467, 41101948, 40945868, 41133284, 43138383, 36781983, 35742563, 41021227, 43215246, 1712891, 1712912, 40903206, 42874705, 44170738, 44178254, 41101949, 44185708, 44178253, 21105226, 41101950, 44185709, 40945869, 41164638, 41320316, 41008133, 41008134, 21134573, 36263434, 40976777, 37593606, 44127082, 41021228, 21115056, 35861961, 35861962, 1592273, 1592275, 36781988, 36781986, 36781985, 43022221, 36781984, 36506063, 36508891, 36781987, 1592274, 1592276, 36419263, 40720770, 40720769, 782277, 40720771, 36508857, 36507001, 36419264, 40720772, 36246126, 42705414, 42705416, 43215245, 43193382, 1718276, 1718277, 1718278, 40720774, 40720773, 44164477, 44168401, 36246120, 21134572, 21124654, 40143571, 40143572, 36246136), 1, 0),
    daysSupplyLessThan30_binary = ifelse(days_supply < 30 | is.na(days_supply), 1, 0)
  )

head(pep2)

In [ ]:
# Make column for drug exposure start date as date
pep2 <- pep2 %>% mutate(drug_exposure_start_date = as.Date(drug_exposure_start_datetime))

head(pep2)

In [ ]:
# Make binary flags for truvada and PEP integrase inhibitor for each date per person id
pep3 <- pep2 %>% group_by(person_id, drug_exposure_start_date, days_supply) %>% summarise(truvada_binary2 = max(truvada_binary),
                                                                            PEP_integrase_inhibitor_binary2 = max(PEP_integrase_inhibitor_binary))

head(pep3)

In [ ]:
# Filter on those who have truvada and PEP integrase inhibitor
pep4 <- pep3 %>% filter(truvada_binary2 == 1 & PEP_integrase_inhibitor_binary2 == 1)

head(pep4)

In [ ]:
# Make binary flag for days supply < 30 or null
pep4 <- pep4 %>% mutate(daysSupplyLessThan30_binary = ifelse(days_supply < 30 | is.na(days_supply),1,0))

head(pep4)

In [ ]:
# Filter on days supply < 30
pep5 <- pep4 %>% filter(daysSupplyLessThan30_binary == 1)

head(pep5)

In [ ]:
# Count number of rows per person id
pep6 <- pep5 %>% group_by(person_id) %>% summarise(row_count = n())

head(pep6)

In [ ]:
# Filter on individuals with 1 row
pep7 <- pep6 %>% filter(row_count == 1)

head(pep7)
length(unique(pep7$person_id))

# Ritonavir phenotyping

In [ ]:
# Filter on individuals in HIV cohort by drugs only
rit1 <- hiv_combine2 %>% filter(indicator2 == "drug")

head(rit1)

In [ ]:
# Merge to get all HIV drug records
rit2 <- merge(rit1, hiv_hiv_drugs, by = "person_id")

head(rit2)

In [ ]:
# Filter on individuals only taking ritonavir and nothing else

# Define the ritonavir concept IDs
ritonavir_concept_ids <- c(41257682, 41209226, 40724596, 40709680, 42918908, 
                         21043415, 43786180, 35860837, 40996625, 41404350,
                         21072940, 36404012, 35860452, 43219711, 42918909,
                         21092413, 41234056, 41083974, 41013502, 43290082,
                         1592433, 40945667, 43193750, 40080338, 35414451,
                         41302143, 35860458, 19088562, 36277450, 43182790,
                         43732231, 35860840, 44166954, 41226850, 40945664,
                         21121798, 41277370, 40996626, 40171778, 1592435,
                         44101047, 2031631, 40724591, 41404313, 40724604,
                         44036307, 41177701, 19082373, 40724595, 40945669,
                         36412246, 36269549, 35860836, 40883376, 35860455,
                         1592436, 1748960, 40724600, 40852101, 40080336,
                         41101768, 36412245, 41027628, 36783255, 35408398,
                         41039028, 35410525, 19038785, 40945666, 41133092,
                         1748957, 1592434, 40709688, 40724598, 35860454,
                         40724594, 1748959, 41246494, 43171801, 40709689,
                         36219736, 21069606, 21072939, 36219737, 36271562,
                         43153881, 43142737, 41404433, 43696348, 40171781,
                         43768138, 35410716, 36509777, 36262103, 43193749,
                         43678527, 43035797, 43175926, 43160894, 41195801,
                         43660518, 1748982, 40834350, 36407130, 1748954,
                         1592438, 41070325, 21112108, 40883375, 41058809,
                         40914288, 43138758, 36412247, 40171779, 41404332,
                         40883372, 1748921, 41226851, 2031629, 36272735,
                         41257683, 41007937, 19038784, 40724590, 40080337,
                         43840342, 35748013, 40865477, 35860451, 36219735,
                         43276253, 21079385, 43153882, 35412773, 43193748,
                         36275334, 40724599, 40080334, 40883373, 35860453,
                         41404462, 44183594, 35860838, 40724589, 43149674,
                         43215643, 41195800, 43858446, 42876172, 41404340,
                         44033885, 21082601, 43042972, 36406313, 40724601,
                         21043414, 40724597, 41226852, 44180502, 41404281,
                         43208695, 41133093, 41320140, 43149675, 35412557,
                         43208697, 41133091, 35410429, 40883374, 41007936,
                         44062430, 40709681, 41058810, 40220792, 40945668,
                         44174454, 44185672, 2031630, 40724602, 21131573,
                         35860456, 44127029, 35789094, 995126, 40852102,
                         40724592, 35860457, 36409400, 35742186, 1748953,
                         43171800, 40171780, 40896672, 43642381, 40724593,
                         35412885, 40945665, 19016698, 43786182, 36412248,
                         43714174, 36277107, 41101769, 1748984, 43257505,
                         41288745, 41288746, 41184181, 36504619, 43786181)

# Step 1: Get all person IDs who have any drug concept
all_persons_drugs <- rit2 %>%
  group_by(person_id) %>%
  summarize(
    has_ritonavir = any(drug_concept_id %in% ritonavir_concept_ids),
    has_other_drugs = any(!(drug_concept_id %in% ritonavir_concept_ids))
  )

# Step 2: Filter for persons who ONLY have ritonavir (has ritonavir but no other drugs)
ritonavir_only_persons <- all_persons_drugs %>%
  filter(has_ritonavir == TRUE & has_other_drugs == FALSE) %>%
  select(person_id)

# Step 3: Filter the original dataset to only include these persons
rit3 <- rit2 %>%
  inner_join(ritonavir_only_persons, by = "person_id")

head(rit3)

In [ ]:
# Make column for drug exposure start date as date
rit3 <- rit3 %>% mutate(drug_exposure_start_date = as.Date(drug_exposure_start_datetime))

head(rit3)

In [ ]:
# Filter on ritonavir use after 1/1/2020
rit4 <- rit3 %>% filter(drug_exposure_start_date > "2020-01-01")

head(rit4)

In [ ]:
# Look at drug source value column
summary(as.factor(rit4$drug_source_value))

*Note, in N3C, there were additional indicators of indivdiuals taking ritonavir/paxlovid but not seeing anything in AoU*

In [ ]:
# Create table of distinct person id
rit_final2 <- rit4 %>% distinct(person_id)

head(rit_final2)
length(unique(rit_final2$person_id))

# Continue HIV phenotyping

In [ ]:
# Exclude individuals in PEP and ritonavir cohorts
hiv_combine6 <- hiv_combine5 %>% filter(!person_id %in% pep7$person_id)
hiv_combine7 <- hiv_combine6 %>% filter(!person_id %in% rit_final2$person_id)

head(hiv_combine7)
nrow(hiv_combine7)
nrow(hiv_combine6)
nrow(hiv_combine5)

In [ ]:
# Filter on patients in ritonavir cohort
hiv_rit <- merge(hiv_combine7, rit3[, !colnames(rit3) %in% "indicator2"], by = "person_id")

# Make column for confidence level
hiv_rit$confidence_level <- 4

head(hiv_rit)

In [ ]:
# Exclude individuals in ritonavir cohort
hiv_combine8 <- hiv_combine7 %>% filter(!person_id %in% rit3$person_id)

head(hiv_combine8)
nrow(hiv_combine8)
nrow(hiv_combine7)

In [ ]:
# Make column for confidence level

# level 1 (highest confidence): 
#### 1) HIV VL PCR >50 copies/mL 
#### 2) HIV VL repeated, regardless of result, at least twice that are at least 90 days apart
#### 3) HIV genotype done, regardless of result
#### 4) HIV screening Ab/Ag test result as positive
#### 5) (Condition OR drug) AND (HIV VL OR CD4 test done, regardless of result)
#### 6) (Condition) AND (2+ occurrences of ARVs for HIV treatment drugs that are at least one day apart) 

# level 2: 
#### 7) (Condition) AND (drug) AND (any of following HIV-related lab test regardless of result: HIV VL, genotype, CD4 test)
#### 8) Any combination of two of the following three: (condition) OR (drug) OR (any HIV-related lab test (e.g., HIV VL>50 copies/mL, CD4 test done regardless of result, HIV screening Ab/Ag test))
#### 9)Condition only for 2+ occurrences that are at least 1 days apart

# level 3:
#### 10) (Condition OR drug) AND (any HIV-related lab test regardless of result (e.g., HIV VL, genotype, CD4 test done, HIV screening Ab/Ag test)) 
#### 11) Drugs only for 2+ occurrences that are at least 1 day apart
#### 12) Drugs only for all existing observations only on 1 date (excluding ritonavir only for possible Paxlovid exposure
#### 13) 1 condition only*

# level 4 (lowest confidence):
#### 14) Ritonavir only for all existing observations, regardless of the number of days of the observation*

hiv_combine8 <- hiv_combine8 %>%
  mutate(confidence_level = case_when(
    # Level 1
    vl_gr50_binary == 1 ~ 1,  # 1)
    (vl_count_measurements >= 2 & vl_days_between_first_last_measurement > 90) ~ 1,  # 2)
    hiv_genotype_binary == 1 ~ 1,  # 3)
    hiv_screen_positive_binary == 1 ~ 1,  # 4)
    (hiv_pt_condition_binary == 1 & vl_count_measurements >= 1) ~ 1,  # 5)
    (hiv_pt_condition_binary == 1 & cd4_count_measurements >= 1) ~ 1,  # 5)
    (hiv_pt_drug_binary == 1 & vl_count_measurements >= 1) ~ 1,  # 5)
    (hiv_pt_drug_binary == 1 & cd4_count_measurements >= 1) ~ 1,  # 5)
    (hiv_pt_condition_binary == 1 & drug_count >= 2 & days_between_first_last_drug_exp >= 1) ~ 1,  # 6)
    
    # Level 2
    (hiv_pt_condition_binary == 1 & hiv_pt_drug_binary == 1 & any_hiv_lab_count >= 1) ~ 1,  # 7)
    indicator2 %in% c('condition and lab', 'condition and drug', 'lab and condition', 
                           'lab and drug', 'drug and condition', 'drug and lab', 
                           'condition and lab and drug', 'condition and drug and lab', 
                           'lab and condition and drug', 'lab and drug and condition', 
                           'drug and condition and lab', 'drug and lab and condition') ~ 2,  # 8)
    (indicator2 %in% c('condition') & condition_count >= 2 & 
      condition_days_between_first_last_dx >= 1) ~ 2,  # 9)
    
    # Level 3
    (hiv_pt_condition_binary == 1 & any_hiv_lab_count >= 1) ~ 2,  # 10)
    (hiv_pt_drug_binary == 1 & any_hiv_lab_count >= 1) ~ 2,  # 10)
    (indicator2 %in% c('drug') & drug_count >= 2 & 
      days_between_first_last_drug_exp >= 1) ~ 3,  # 11)
    (indicator2 %in% c('drug') & days_between_first_last_drug_exp == 0) ~ 3,  # 12)
    (indicator2 %in% c('condition') & condition_days_between_first_last_dx == 0) ~ 3,  # 13)
    
    # Default case
    TRUE ~ 0
  ))

head(hiv_combine8)

In [ ]:
# Filter on columns of interest for rbind below
hiv_rit2 <- hiv_rit[,c(1:18,46)]

colnames(hiv_combine8)
colnames(hiv_rit2)
ncol(hiv_combine8)
ncol(hiv_rit2)

In [ ]:
# Join individuals from confidence levels 1-3 and 4
hiv_combine9 <- rbind(hiv_combine8, hiv_rit2)

head(hiv_combine9)

In [ ]:
# Make column for criterion number

# level 1 (highest confidence): 
#### 1) HIV VL PCR >50 copies/mL 
#### 2) HIV VL repeated, regardless of result, at least twice that are at least 90 days apart
#### 3) HIV genotype done, regardless of result
#### 4) HIV screening Ab/Ag test result as positive
#### 5) (Condition OR drug) AND (HIV VL OR CD4 test done, regardless of result)
#### 6) (Condition) AND (2+ occurrences of ARVs for HIV treatment drugs that are at least one day apart) 

# level 2: 
#### 7) (Condition) AND (drug) AND (any of following HIV-related lab test regardless of result: HIV VL, genotype, CD4 test)
#### 8) Any combination of two of the following three: (condition) OR (drug) OR (any HIV-related lab test (e.g., HIV VL>50 copies/mL, CD4 test done regardless of result, HIV screening Ab/Ag test))
#### 9)Condition only for 2+ occurrences that are at least 1 days apart

# level 3:
#### 10) (Condition OR drug) AND (any HIV-related lab test regardless of result (e.g., HIV VL, genotype, CD4 test done, HIV screening Ab/Ag test)) 
#### 11) Drugs only for 2+ occurrences that are at least 1 day apart
#### 12) Drugs only for all existing observations only on 1 date (excluding ritonavir only for possible Paxlovid exposure
#### 13) 1 condition only*

# level 4 (lowest confidence):
#### 14) Ritonavir only for all existing observations, regardless of the number of days of the observation*

hiv_combine9 <- hiv_combine9 %>%
  mutate(criteria_count = case_when(
    # Level 1
    vl_gr50_binary == 1 ~ 1,  # 1)
    (vl_count_measurements >= 2 & vl_days_between_first_last_measurement > 90) ~ 2,  # 2)
    hiv_genotype_binary == 1 ~ 3,  # 3)
    hiv_screen_positive_binary == 1 ~ 4,  # 4)
    (hiv_pt_condition_binary == 1 & vl_count_measurements >= 1) ~ 5,  # 5)
    (hiv_pt_condition_binary == 1 & cd4_count_measurements >= 1) ~ 5,  # 5)
    (hiv_pt_drug_binary == 1 & vl_count_measurements >= 1) ~ 5,  # 5)
    (hiv_pt_drug_binary == 1 & cd4_count_measurements >= 1) ~ 5,  # 5)
    (hiv_pt_condition_binary == 1 & drug_count >= 2 & days_between_first_last_drug_exp >= 1) ~ 6,  # 6)
    
    # Level 2
    (hiv_pt_condition_binary == 1 & hiv_pt_drug_binary == 1 & any_hiv_lab_count >= 1) ~ 7,  # 7)
    indicator2 %in% c('condition and lab', 'condition and drug', 'lab and condition', 
                           'lab and drug', 'drug and condition', 'drug and lab', 
                           'condition and lab and drug', 'condition and drug and lab', 
                           'lab and condition and drug', 'lab and drug and condition', 
                           'drug and condition and lab', 'drug and lab and condition') ~ 8,  # 8)
    (indicator2 %in% c('condition') & condition_count >= 2 & 
      condition_days_between_first_last_dx >= 1) ~ 9,  # 9)
    
    # Level 3
    (hiv_pt_condition_binary == 1 & any_hiv_lab_count >= 1) ~ 10,  # 10)
    (hiv_pt_drug_binary == 1 & any_hiv_lab_count >= 1) ~ 10,  # 10)
    (indicator2 %in% c('drug') & drug_count >= 2 & 
      days_between_first_last_drug_exp >= 1) ~ 11,  # 11)
    (indicator2 %in% c('drug') & days_between_first_last_drug_exp == 0) ~ 12,  # 12)
    (indicator2 %in% c('condition') & condition_days_between_first_last_dx == 0) ~ 13,  # 13)
      
    # Level 4
      confidence_level == 4 ~ 14, #14)
    
    # Default case
    TRUE ~ 0
  ))

head(hiv_combine9)
summary(as.factor(hiv_combine9$criteria_count))

In [ ]:
# Identify individuals in criteria 11 and 12 with HBV dx

# Filter on those in criteria 11 and 12
hiv_combine9_hbv <- hiv_combine9 %>% filter(criteria_count %in% c(11,12))

# Filter on those with HBV dx
hiv_combine9_hbv2 <- hiv_combine9_hbv %>% filter(person_id %in% hbv$person_id)

# Exclude individuals with HBV in criteria 11 and 12
hiv_combine10 <- hiv_combine9 %>% filter(!person_id %in% hiv_combine9_hbv2$person_id)

head(hiv_combine10)
length(unique(hiv_combine10$person_id))
length(unique(hiv_combine9$person_id))

In [ ]:
# Exclude individuals in criterion 7 only on PrEP drugs

# Filter on criterion 7
hiv_combine9_prep <- hiv_combine9 %>% filter(criteria_count == 7)

# Filter on those only using PrEP
hiv_combine9_prep2 <- hiv_combine9_prep %>% filter(person_id %in% prep$person_id)

# Exclude those in criteria 7 only on PrEP
hiv_combine11 <- hiv_combine10 %>% filter(!person_id %in% hiv_combine9_prep2$person_id)

head(hiv_combine11)
length(unique(hiv_combine11$person_id))
length(unique(hiv_combine10$person_id))

In [ ]:
# Get all HIV labs except for CD4 and VL for HIV cohort
hiv_combine9_labs <- merge(hiv_hiv_labs, hiv_combine9, by = "person_id")

head(hiv_combine9_labs)

In [ ]:
# Remove individuals with negative AbAg test
neg_abag <- hiv_combine9_labs %>% 
  filter(lab_test_type == "AbAg") %>%
  group_by(person_id) %>%
  summarize(has_positive = any(lab_result == "Positive")) %>%
  filter(!has_positive) %>%
  semi_join(
    hiv_combine9_labs %>% 
      filter(lab_test_type == "AbAg", lab_result == "Negative"),
    by = "person_id"
  ) %>%
  distinct(person_id)

# Identify individuals confidence level 1 in neg_abag
neg_abag2 <- merge(hiv_combine9, neg_abag, by = "person_id")



head(neg_abag2)

In [ ]:
# Remove individuals with negative AbAg test from hiv cohort
hiv_combine12 <- hiv_combine11 %>% filter(!person_id %in% neg_abag2$person_id)

head(hiv_combine12)
length(unique(hiv_combine12$person_id))
length(unique(hiv_combine11$person_id))

# Create group of individuals to add back to HIV cohort

In [ ]:
# Filter on individuals confidence level 1
neg_abag2_conf1 <- neg_abag2 %>% filter(confidence_level == 1)

head(neg_abag2_conf1)

In [ ]:
# Add individuals with genotype to HIV cohort
hiv_labs2_genotype <- hiv_labs2 %>% filter(lab_test_type == "Genotype")

head(hiv_labs2_genotype)
length(unique(hiv_labs2_genotype$person_id))

## Add individuals with VL >=50 copies /mL currently in HIV negative cohort

In [ ]:
# Query for all individuals in AoU

library(tidyverse)
library(bigrquery)

# This query represents dataset "All person table" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_27520316_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth,
        person.self_reported_category_concept_id,
        p_self_reported_category_concept.concept_name as self_reported_category 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id 
    LEFT JOIN
        `concept` p_self_reported_category_concept 
            ON person.self_reported_category_concept_id = p_self_reported_category_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_27520316_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_27520316",
  "person_27520316_*.csv")
message(str_glue('The data will be written to {person_27520316_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_27520316_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_27520316_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_27520316_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character(), self_reported_category = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_27520316_person_df <- read_bq_export_from_workspace_bucket(person_27520316_path)

dim(dataset_27520316_person_df)

head(dataset_27520316_person_df, 5)

In [ ]:
# Create df for HIV negative details
hiv_neg <- dataset_27520316_person_df %>% filter(!person_id %in% hiv_combine12,
                                                !person_id %in% pep7$person_id,
                                                !person_id %in% prep_combine3$person_id)

head(hiv_neg)
length(unique(hiv_neg$person_id))
length(unique(dataset_27520316_person_df$person_id))

In [ ]:
# Prep VL df to get vl category per person 

# Duplicate vl df
df_vl2 <- df_vl

# Make column for measurement_date
df_vl2 <- df_vl2 %>% mutate(measurement_date = as.Date(measurement_datetime))

df_vl_person_category <- df_vl2 %>% 
  group_by(person_id) %>% 
  summarise(
    vl_gr50_binary = max(ifelse(vl_group == 'VL_gr50', 1, 0)),
    vl_less50_binary = max(ifelse(vl_group == 'VL_less50', 1, 0)),
    vl_unknown_binary = max(ifelse(vl_group == 'VL_unknown', 1, 0)),
    vl_measurement_count = n_distinct(measurement_date),
    .groups = 'drop'
  )

# Make column for vl_category
df_vl_person_category <- df_vl_person_category %>% 
  mutate(
    vl_category = case_when(
      vl_gr50_binary == 1 ~ 'vl_gr50',
      vl_gr50_binary == 0 & vl_less50_binary == 1 ~ 'vl_less50',
      vl_gr50_binary == 0 & vl_less50_binary == 0 & vl_unknown_binary == 1 ~ 'vl_unknown'
    )
  )

head(df_vl_person_category)
nrow(df_vl_person_category)
length(unique(df_vl_person_category$person_id))

In [ ]:
# Merge with vl
hiv_neg_hiv_details <- merge(hiv_neg, df_vl_person_category, by = "person_id", all.x = TRUE)

head(hiv_neg_hiv_details)
length(unique(hiv_neg_hiv_details$person_id))
summary(as.factor(hiv_neg_hiv_details$vl_category))

In [ ]:
# Prep CD4 absolute df for QC 

# Duplicate vl df
df_cd4_absolute2 <- df_cd4_absolute

# Make column for measurement_date
df_cd4_absolute2 <- df_cd4_absolute2 %>% mutate(measurement_date = as.Date(measurement_datetime))

df_cd4_person_category <- df_cd4_absolute2 %>% 
  group_by(person_id) %>% 
  summarise(
    cd4_less200_binary = max(ifelse(cd4_group == 'CD4_less200', 1, 0)),
    cd4_gr200_binary = max(ifelse(cd4_group == 'CD4_gr200', 1, 0)),
    cd4_unknown_binary = max(ifelse(cd4_group == 'CD4_unknown', 1, 0)),
    cd4_measurement_count = n_distinct(measurement_date),
    .groups = 'drop'
  )

# Make column for vl_category
df_cd4_person_category <- df_cd4_person_category %>% 
  mutate(
    cd4_category = case_when(
      cd4_less200_binary == 1 ~ 'cd4_less200',
      cd4_less200_binary == 0 & cd4_gr200_binary == 1 ~ 'cd4_gr200',
      cd4_less200_binary == 0 & cd4_gr200_binary == 0 & cd4_unknown_binary == 1 ~ 'cd4_unknown'
    )
  )

head(df_cd4_person_category)
nrow(df_cd4_person_category)
length(unique(df_cd4_person_category$person_id))

In [ ]:
# Merge with cd4
hiv_neg_hiv_details2 <- merge(hiv_neg_hiv_details, df_cd4_person_category, by = "person_id", all.x = TRUE)

head(hiv_neg_hiv_details2)
length(unique(hiv_neg_hiv_details2$person_id))

In [ ]:
# Exclude individuals with VL >= 50 copies/mL
hiv_neg_vl_gr50 <- hiv_neg_hiv_details2 %>% filter(vl_category == "vl_gr50")

# Exclude
hiv_neg2 <- hiv_neg_hiv_details2 %>% filter(!person_id %in% hiv_neg_vl_gr50$person_id)

head(hiv_neg2)
length(unique(hiv_neg2$person_id))
length(unique(hiv_neg$person_id))

In [ ]:
# Remove individuals who are confidence level 1 from negative AbAg test
hiv_neg3 <- hiv_neg2 %>% filter(!person_id %in% neg_abag2_conf1$person_id)

head(hiv_neg3)
length(unique(hiv_neg3$person_id))
length(unique(hiv_neg2$person_id))

In [ ]:
# Remove individuals with genotype test
hiv_neg4 <- hiv_neg3 %>% filter(!person_id %in% hiv_labs2_genotype$person_id)

head(hiv_neg4)
length(unique(hiv_neg4$person_id))
length(unique(hiv_neg3$person_id))

In [ ]:
# Set up group of individuals to add to hiv cohort
add_to_hiv_cohort <- hiv_neg4 %>% filter(!is.na(vl_category) & !is.na(cd4_category))

head(add_to_hiv_cohort)
length(unique(add_to_hiv_cohort$person_id))

In [ ]:
# Get individuals in HIV negative with VL gr50
hiv_neg_vl_gr50 <- hiv_neg_hiv_details2 %>% filter(vl_group == "VL_gr50")

head(hiv_neg_vl_gr50)
length(unique(hiv_neg_vl_gr50$person_id))

In [ ]:
# Create column for confidence level of those being added back to hiv cohort

# VL and Cd4 present but no results
add_to_hiv_cohort$confidence_level <- 2

# Genotype - level 1
hiv_labs2_genotype$confidence_level <- 1

# Neg AbAg but confidence level was 1 before being excluded
neg_abag2_conf1$confidence_level <- 1

# In hiv negative but VL >= 50 copies/mL
hiv_neg_vl_gr50$confidence_level <- 1

print("complete")

In [ ]:
length(unique(add_to_hiv_cohort$person_id))
length(unique(hiv_labs2_genotype$person_id))
length(unique(neg_abag2_conf1$person_id))
length(unique(hiv_neg_vl_gr50$person_id))

In [ ]:
# Continue setting up individuals to add to hiv cohort
add_to_hiv_cohort2 <- rbind(add_to_hiv_cohort %>% distinct(person_id, confidence_level), hiv_labs2_genotype %>% distinct(person_id, confidence_level), neg_abag2_conf1 %>% distinct(person_id, confidence_level), hiv_neg_vl_gr50 %>% distinct(person_id, confidence_level))

head(add_to_hiv_cohort2)
nrow(add_to_hiv_cohort2)

# Continue processing HIV cohort

In [ ]:
# Continue setting up individuals to add to hiv cohort
add_to_hiv_cohort2 <- rbind(add_to_hiv_cohort %>% distinct(person_id, confidence_level), hiv_labs2_genotype %>% distinct(person_id, confidence_level), neg_abag2_conf1 %>% distinct(person_id, confidence_level), hiv_neg_vl_gr50 %>% distinct(person_id, confidence_level))

head(add_to_hiv_cohort2)
nrow(add_to_hiv_cohort2)

In [ ]:
# Add individuals to hiv cohort
hiv_cohort <- rbind(hiv_combine12 %>% distinct(person_id, confidence_level), add_to_hiv_cohort2)

head(hiv_cohort)
length(unique(hiv_cohort$person_id))

In [ ]:
# Remove confidence level 4
hiv_cohort2 <- hiv_cohort %>% filter(confidence_level != 4)

head(hiv_cohort2)
length(unique(hiv_cohort2$person_id))

In [ ]:
# Select the highest confidence level per person id
hiv_cohort3 <- hiv_cohort2 %>% group_by(person_id) %>% summarise(confidence_level = min(confidence_level))

head(hiv_cohort3)
nrow(hiv_cohort3)
length(unique(hiv_cohort3$person_id))

In [ ]:
# Get counts of each confidence level
hiv_cohort3 %>% group_by(confidence_level) %>% summarise(Count_person_id = n_distinct(person_id))

*not sure what confidence level 0 is so will need to revise a bit in the future figure out which group they belong to*

# Quality control

## Set up cohorts

In [ ]:
# Prep VL df for QC 

# Duplicate vl df
df_vl2 <- df_vl

# Make column for measurement_date
df_vl2 <- df_vl2 %>% mutate(measurement_date = as.Date(measurement_datetime))

df_vl_person_category <- df_vl2 %>% 
  group_by(person_id) %>% 
  summarise(
    vl_gr50_binary = max(ifelse(vl_group == 'VL_gr50', 1, 0)),
    vl_less50_binary = max(ifelse(vl_group == 'VL_less50', 1, 0)),
    vl_unknown_binary = max(ifelse(vl_group == 'VL_unknown', 1, 0)),
    vl_measurement_count = n_distinct(measurement_date),
    .groups = 'drop'
  )

# Make column for vl_category
df_vl_person_category <- df_vl_person_category %>% 
  mutate(
    vl_category = case_when(
      vl_gr50_binary == 1 ~ 'vl_gr50',
      vl_gr50_binary == 0 & vl_less50_binary == 1 ~ 'vl_less50',
      vl_gr50_binary == 0 & vl_less50_binary == 0 & vl_unknown_binary == 1 ~ 'vl_unknown'
    )
  )

head(df_vl_person_category)
nrow(df_vl_person_category)
length(unique(df_vl_person_category$person_id))

In [ ]:
# Prep CD4 absolute df for QC 

# Duplicate vl df
df_cd4_absolute2 <- df_cd4_absolute

# Make column for measurement_date
df_cd4_absolute2 <- df_cd4_absolute2 %>% mutate(measurement_date = as.Date(measurement_datetime))

df_cd4_person_category <- df_cd4_absolute2 %>% 
  group_by(person_id) %>% 
  summarise(
    cd4_less200_binary = max(ifelse(cd4_group == 'CD4_less200', 1, 0)),
    cd4_gr200_binary = max(ifelse(cd4_group == 'CD4_gr200', 1, 0)),
    cd4_unknown_binary = max(ifelse(cd4_group == 'CD4_unknown', 1, 0)),
    cd4_measurement_count = n_distinct(measurement_date),
    .groups = 'drop'
  )

# Make column for vl_category
df_cd4_person_category <- df_cd4_person_category %>% 
  mutate(
    cd4_category = case_when(
      cd4_less200_binary == 1 ~ 'cd4_less200',
      cd4_less200_binary == 0 & cd4_gr200_binary == 1 ~ 'cd4_gr200',
      cd4_less200_binary == 0 & cd4_gr200_binary == 0 & cd4_unknown_binary == 1 ~ 'cd4_unknown'
    )
  )

head(df_cd4_person_category)
nrow(df_cd4_person_category)
length(unique(df_cd4_person_category$person_id))

In [ ]:
# Filter on confidence levels 1 and 2
hiv_cohort12 <- hiv_cohort3 %>% filter(confidence_level %in% c(1,2))

# Get HIV details
hiv_cohort12_hiv_1 <- merge(hiv_cohort12 %>% distinct(person_id, confidence_level), df_vl_person_category, by = "person_id", all.x = TRUE)
hiv_cohort12_hiv_2 <- merge(hiv_cohort12_hiv_1, df_cd4_person_category, by = "person_id", all.x = TRUE)

head(hiv_cohort12_hiv_2)
nrow(hiv_cohort12_hiv_2)
length(unique(hiv_cohort12_hiv_2$person_id))

In [ ]:
# HIV cohort confidence levels 1,2,3
hiv_cohort123 <- hiv_cohort3

# Get HIV details
hiv_cohort123_hiv_1 <- merge(hiv_cohort123, df_vl_person_category, by = "person_id", all.x = TRUE)
hiv_cohort123_hiv_2 <- merge(hiv_cohort123_hiv_1, df_cd4_person_category, by = "person_id", all.x = TRUE)

head(hiv_cohort123_hiv_2)
length(unique(hiv_cohort123_hiv_2$person_id))

In [ ]:
# prep cohort
prep_cohort <- prep_combine3

# Get HIV details
prep_cohort_hiv_1 <- merge(prep_cohort, df_vl_person_category, by = "person_id", all.x = TRUE)
prep_cohort_hiv_2 <- merge(prep_cohort_hiv_1, df_cd4_person_category, by = "person_id", all.x = TRUE)

head(prep_cohort_hiv_2)

In [ ]:
# pep cohort
pep_cohort <- pep7

# Get HIV details
pep_cohort_hiv_1 <- merge(pep_cohort, df_vl_person_category, by = "person_id", all.x = TRUE)
pep_cohort_hiv_2 <- merge(pep_cohort_hiv_1, df_cd4_person_category, by = "person_id", all.x = TRUE)

head(pep_cohort_hiv_2)

In [ ]:
# hiv negative

# Filter on columns of interest
hiv_negative_cohort <- hiv_neg4 %>% select(all_of(names(pep_cohort_hiv_2 %>% select(-row_count))))
head(hiv_negative_cohort)
nrow(hiv_negative_cohort)
length(unique(hiv_negative_cohort$person_id))

## Run QC analyses

In [ ]:
# Filter on columns of interest
hiv_cohort12_hiv_3 <- hiv_cohort12_hiv_2 %>% select(-confidence_level)
hiv_cohort123_hiv_3 <- hiv_cohort123_hiv_2 %>% select(-confidence_level)
prep_cohort_hiv_3 <- prep_cohort_hiv_2 %>% select(-c(indicator2, final_lab_result2, confidence_level))
pep_cohort_hiv_2 <- pep_cohort_hiv_2 %>% select(-row_count)

# Add column for cohort
hiv_cohort12_hiv_3$cohort <- "hiv_conf12"
hiv_cohort123_hiv_3$cohort <- "hiv_conf123"
prep_cohort_hiv_3$cohort <- "prep"
pep_cohort_hiv_2$cohort <- "pep"
hiv_negative_cohort$cohort <- "hiv_negative"

colnames(hiv_cohort12_hiv_3)
colnames(hiv_cohort123_hiv_3)
colnames(prep_cohort_hiv_3)
colnames(pep_cohort_hiv_2)
colnames(hiv_negative_cohort)

In [ ]:
# Make column for where tables came from
df_combine <- rbind(hiv_cohort12_hiv_3, hiv_cohort123_hiv_3, prep_cohort_hiv_3, pep_cohort_hiv_2, hiv_negative_cohort)

head(df_combine)

In [ ]:
# At least 1 vl measurement
df_combine %>% group_by(cohort) %>% filter(vl_measurement_count >= 1) %>% summarise(Count_person_id = n_distinct(person_id))

# Count per vl category
df_combine %>% group_by(cohort,vl_category) %>% summarise(Count_person_id = n_distinct(person_id))

# Count per cd4 category
df_combine %>% group_by(cohort,cd4_category) %>% summarise(Count_person_id = n_distinct(person_id))

# Median (Q1-Q3) number of vl measurements
df_combine %>% 
  group_by(cohort) %>% 
  summarise(
    quantile_vl_measurement_count = quantile(na.omit(vl_measurement_count)),
    quantile_label = c("0%", "25%", "50%", "75%", "100%"),
    .groups = 'drop'
  )

# Median (Q1-Q3) number of cd4 measurements
df_combine %>% 
  group_by(cohort) %>% 
  summarise(
    quantile_cd4_measurement_count = quantile(na.omit(cd4_measurement_count)),
    quantile_label = c("0%", "25%", "50%", "75%", "100%"),
    .groups = 'drop'
  )